In [8]:
import numpy as np
from collections import Counter

# =========================
# 1. 加载原始数据
# =========================
poses = np.load("data/poses_19MM.npy")          # (33955, 32, 3)
labels = np.load("data/frame_labels.npy", allow_pickle=True)  # (34001,)

print("原始骨架 shape:", poses.shape)
print("原始标签 shape:", labels.shape)

# =========================
# 2. 对齐长度
# =========================
min_len = min(len(poses), len(labels))
poses = poses[:min_len]
labels = labels[:min_len]

print("\n对齐后骨架 shape:", poses.shape)
print("对齐后标签 shape:", labels.shape)

# =========================
# 3. 检查类别分布
# =========================
counts = Counter(labels)
print("\n标签分布（前10个类别）:")
for k, v in list(counts.items())[:10]:
    print(f"{k}: {v}")
print("总类别数:", len(counts))

# =========================
# 4. 去掉 'Unknown'
# =========================
mask = labels != "Unknown"
poses_clean = poses[mask]
labels_clean = labels[mask]

print("\n清理后骨架 shape:", poses_clean.shape)
print("清理后标签 shape:", labels_clean.shape)


原始骨架 shape: (33955, 32, 3)
原始标签 shape: (34001,)

对齐后骨架 shape: (33955, 32, 3)
对齐后标签 shape: (33955,)

标签分布（前10个类别）:
Unknown: 4807
a_walk: 4333
p_stand: 7965
t_stand_to_sit: 324
p_sit: 10054
t_sit_to_stand: 216
t_sit_to_lie: 150
p_lie: 2157
t_bed_turn: 485
t_lie_to_situp: 67
总类别数: 16

清理后骨架 shape: (29148, 32, 3)
清理后标签 shape: (29148,)


In [16]:
import numpy as np
from collections import Counter

# =========================
# 1. 加载原始数据
# =========================
poses = np.load("data/poses_19MM.npy")          # (33955, 32, 3)
labels = np.load("data/frame_labels.npy", allow_pickle=True)  # (34001,)

print("原始骨架 shape:", poses.shape)
print("原始标签 shape:", labels.shape)

# =========================
# 2. 对齐长度
# =========================
min_len = min(len(poses), len(labels))
poses = poses[:min_len]
labels = labels[:min_len]

print("\n对齐后骨架 shape:", poses.shape)
print("对齐后标签 shape:", labels.shape)

# =========================
# 3. 去掉 'Unknown'
# =========================
mask = labels != "Unknown"
poses_clean = poses[mask]
labels_clean = labels[mask]

print("\n清理后骨架 shape:", poses_clean.shape)
print("清理后标签 shape:", labels_clean.shape)

# =========================
# 4. 滑窗切分 (每个样本 15 帧)
# =========================
# ======================
# 切分窗口 (改为 19 帧)
# ======================
window = 19
stride = 19   # 或者用更小 stride 来增加样本数量，例如 stride=10

X_samples = []
y_samples = []

for start in range(0, len(poses_clean) - window + 1, stride):
    end = start + window
    X_samples.append(poses_clean[start:end])  # (19, 32, 3)
    mid = start + window // 2
    y_samples.append(labels_clean[mid])       # 取中间帧标签

X_samples = np.array(X_samples)  # (N, 19, 32, 3)
y_samples = np.array(y_samples)

# reshape 成 (N, 19, 96) 给 LSTM 用
X_samples = X_samples.reshape(X_samples.shape[0], window, -1)

print("最终 X shape:", X_samples.shape)
print("最终 y shape:", y_samples.shape)

# 保存结果
np.save("data/X_lstm.npy", X_samples)
np.save("data/y_lstm.npy", y_samples)


print("\n✅ 已保存到 data/X_lstm.npy 和 data/y_lstm.npy")


原始骨架 shape: (33955, 32, 3)
原始标签 shape: (34001,)

对齐后骨架 shape: (33955, 32, 3)
对齐后标签 shape: (33955,)

清理后骨架 shape: (29148, 32, 3)
清理后标签 shape: (29148,)
最终 X shape: (1534, 19, 96)
最终 y shape: (1534,)

✅ 已保存到 data/X_lstm.npy 和 data/y_lstm.npy


In [17]:
import numpy as np
from sklearn.preprocessing import LabelEncoder

# 读取刚才保存的标签
y = np.load("data/y_lstm.npy", allow_pickle=True)

# 编码成 0,1,2,... 
le = LabelEncoder()
y_encoded = le.fit_transform(y)

print("类别数:", len(le.classes_))
print("类别映射:", dict(zip(le.classes_, range(len(le.classes_)))))

# 保存
np.save("data/y_lstm_encoded.npy", y_encoded)
np.save("data/label_classes.npy", le.classes_)

print("✅ 已保存 data/y_lstm_encoded.npy 和 data/label_classes.npy")


类别数: 15
类别映射: {np.str_('a_walk'): 0, np.str_('p_bent'): 1, np.str_('p_lie'): 2, np.str_('p_sit'): 3, np.str_('p_situp'): 4, np.str_('p_stand'): 5, np.str_('t_bed_turn'): 6, np.str_('t_bend'): 7, np.str_('t_lie_to_sit'): 8, np.str_('t_lie_to_situp'): 9, np.str_('t_sit_to_lie'): 10, np.str_('t_sit_to_stand'): 11, np.str_('t_situp_to_sit'): 12, np.str_('t_stand_to_sit'): 13, np.str_('t_straighten'): 14}
✅ 已保存 data/y_lstm_encoded.npy 和 data/label_classes.npy


In [19]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# =====================
# 1. 加载数据
# =====================
X = np.load("data/X_lstm.npy")        # (N, T, 32, 3) 或 (N, T, 96)
y = np.load("data/y_lstm.npy")        # (N, )

# 如果是 4D 就 reshape 成 (N, T, 96)
if X.ndim == 4:
    N, T, J, C = X.shape
    X = X.reshape(N, T, J * C)

print("修正后 X shape:", X.shape, "y shape:", y.shape)

# 标签编码
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
print("类别数:", num_classes, "类别:", le.classes_)

# 保存类别映射
np.save("data/label_classes.npy", le.classes_)

# =====================
# 2. 划分训练/验证集
# =====================
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 转 Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val   = torch.tensor(y_val, dtype=torch.long)

# DataLoader
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val, y_val), batch_size=32, shuffle=False)

# =====================
# 3. BiLSTM 模型
# =====================
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.5):
        super(BiLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_size,
            hidden_size=hidden_size,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout,
            bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)  # 双向 ×2

    def forward(self, x):
        out, _ = self.lstm(x)   # (batch, seq_len, hidden*2)
        out = out[:, -1, :]     # 取最后时刻
        out = self.fc(out)
        return out

# =====================
# 4. 损失函数 + 优化器 + 学习率调度
# =====================
# 类别权重（按训练集统计）
class_counts = Counter(y_train.numpy())
weights = torch.tensor([1.0 / (class_counts[i] + 1e-6) for i in range(num_classes)],
                       dtype=torch.float32)
weights = weights / weights.sum() * num_classes  # 归一化
criterion = nn.CrossEntropyLoss(weight=weights)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTMClassifier(input_size=X.shape[2], hidden_size=128, num_layers=2,
                         num_classes=num_classes, dropout=0.5).to(device)

optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(
    optimizer, mode='min', factor=0.5, patience=3
)

# =====================
# 5. 训练循环
# =====================
epochs = 30
for epoch in range(epochs):
    # ---- Train ----
    model.train()
    train_loss, correct, total = 0.0, 0, 0
    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X_batch.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == y_batch).sum().item()
        total += y_batch.size(0)

    train_acc = correct / total
    train_loss /= total

    # ---- Validation ----
    model.eval()
    val_loss, correct, total = 0.0, 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            val_loss += loss.item() * X_batch.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

    val_acc = correct / total
    val_loss /= total
    scheduler.step(val_loss)

    print(f"Epoch {epoch+1}/{epochs} | "
          f"Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

# =====================
# 6. 评估
# =====================
model.eval()
y_true, y_pred = [], []
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, preds = torch.max(outputs, 1)
        y_true.extend(y_batch.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

y_true = np.array(y_true)
y_pred = np.array(y_pred)

print("\n分类报告:")
labels = list(range(len(le.classes_)))  # 保证和 target_names 对齐
print(classification_report(
    y_true, y_pred,
    labels=labels,
    target_names=le.classes_,
    zero_division=0
))

print("\n混淆矩阵:")
print(confusion_matrix(y_true, y_pred, labels=labels))

# =====================
# 7. 保存模型
# =====================
torch.save(model.state_dict(), "data/lstm_bilstm_weighted_final.pth")
print("✅ 模型已保存到 data/lstm_bilstm_weighted_final.pth")


修正后 X shape: (1534, 19, 96) y shape: (1534,)
类别数: 15 类别: ['a_walk' 'p_bent' 'p_lie' 'p_sit' 'p_situp' 'p_stand' 't_bed_turn'
 't_bend' 't_lie_to_sit' 't_lie_to_situp' 't_sit_to_lie' 't_sit_to_stand'
 't_situp_to_sit' 't_stand_to_sit' 't_straighten']
Epoch 1/30 | Train Loss: 2.6446 Acc: 0.1361 | Val Loss: 2.5403 Acc: 0.0847
Epoch 2/30 | Train Loss: 2.5267 Acc: 0.1597 | Val Loss: 2.3858 Acc: 0.1238
Epoch 3/30 | Train Loss: 2.4460 Acc: 0.1907 | Val Loss: 2.3910 Acc: 0.2345
Epoch 4/30 | Train Loss: 2.4433 Acc: 0.1280 | Val Loss: 2.3972 Acc: 0.1466
Epoch 5/30 | Train Loss: 2.4055 Acc: 0.1385 | Val Loss: 2.3639 Acc: 0.2117
Epoch 6/30 | Train Loss: 2.4165 Acc: 0.1589 | Val Loss: 2.3466 Acc: 0.2117
Epoch 7/30 | Train Loss: 2.4028 Acc: 0.1785 | Val Loss: 2.3596 Acc: 0.1303
Epoch 8/30 | Train Loss: 2.3588 Acc: 0.1720 | Val Loss: 2.3555 Acc: 0.1531
Epoch 9/30 | Train Loss: 2.3757 Acc: 0.1606 | Val Loss: 2.3834 Acc: 0.1726
Epoch 10/30 | Train Loss: 2.3578 Acc: 0.1850 | Val Loss: 2.3469 Acc: 0.1466

In [23]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
import random

# =============== 数据增强函数 ===============
def augment_skeleton_sequence(seq, noise_std=0.01, rotate=True):
    """
    对单个骨架序列进行增强
    seq: (T, J) or (T, J, C)，时间长度 T，关节数 J，坐标维度 C
    """
    seq_aug = seq.copy()

    # 1. 加入高斯噪声
    seq_aug = seq_aug + np.random.normal(0, noise_std, seq_aug.shape)

    # 2. 随机旋转 (仅绕 Z 轴, 模拟平面旋转)
    if rotate:
        theta = np.random.uniform(-10, 10) * np.pi / 180  # -10° ~ +10°
        cos_t, sin_t = np.cos(theta), np.sin(theta)
        R = np.array([[cos_t, -sin_t, 0],
                      [sin_t,  cos_t, 0],
                      [0,      0,     1]])
        if seq_aug.ndim == 3 and seq_aug.shape[2] >= 3:
            seq_aug[..., :3] = seq_aug[..., :3] @ R.T

    return seq_aug


def balance_and_augment(X, y, min_samples=50):
    """
    平衡并增强数据
    X: (N, T, F) 骨架序列
    y: (N,) 标签
    """
    X_bal, y_bal = [], []
    classes = np.unique(y)
    
    for cls in classes:
        X_cls = X[y == cls]
        y_cls = y[y == cls]

        # 如果类别太少，复制到 min_samples
        if len(X_cls) < min_samples:
            X_res, y_res = resample(
                X_cls, y_cls,
                replace=True,
                n_samples=min_samples,
                random_state=42
            )
        else:
            X_res, y_res = X_cls, y_cls

        # 数据增强
        X_aug = np.array([augment_skeleton_sequence(seq) for seq in X_res])

        X_bal.append(X_aug)
        y_bal.append(y_res)

    X_bal = np.vstack(X_bal)
    y_bal = np.hstack(y_bal)
    return X_bal, y_bal

# =============== BiLSTM 模型 ===============
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, num_layers=1, dropout=0.3):
        super(BiLSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_size, hidden_size, num_layers,
                            batch_first=True, bidirectional=True, dropout=dropout)
        self.fc = nn.Linear(hidden_size*2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)  # (batch, seq_len, hidden*2)
        out = out[:, -1, :]    # 取最后时刻
        out = self.fc(out)
        return out

# =============== 主程序 ===============
if __name__ == "__main__":
    # 固定随机种子，保证复现
    torch.manual_seed(42)
    np.random.seed(42)
    random.seed(42)

    X = np.load("data/X_lstm.npy")
    y = np.load("data/y_lstm.npy")


    print("原始 X shape:", X.shape, "y shape:", y.shape)

    # 2. 标签编码
    le = LabelEncoder()
    y_encoded = le.fit_transform(y)
    num_classes = len(le.classes_)
    print("类别数:", num_classes, "类别:", le.classes_)

    # 3. 样本均衡 + 增强
    X_bal, y_bal = balance_and_augment(X, y_encoded, min_samples=100)
    print("平衡后 X shape:", X_bal.shape, "y shape:", y_bal.shape)

    # 4. 划分训练/验证集
    X_train, X_val, y_train, y_val = train_test_split(
        X_bal, y_bal, test_size=0.2, random_state=42, stratify=y_bal
    )

    # 转 tensor
    X_train = torch.tensor(X_train, dtype=torch.float32)
    y_train = torch.tensor(y_train, dtype=torch.long)
    X_val = torch.tensor(X_val, dtype=torch.float32)
    y_val = torch.tensor(y_val, dtype=torch.long)

    # 5. DataLoader
    batch_size = 32
    train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
    val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size)

    # 6. 模型、优化器、损失函数
    input_size = X.shape[2]
    hidden_size = 128
    model = BiLSTMClassifier(input_size, hidden_size, num_classes, num_layers=2, dropout=0.3)
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model = model.to(device)

    # 加权交叉熵
    class_counts = np.bincount(y_bal)
    weights = torch.tensor(1.0 / (class_counts + 1e-6), dtype=torch.float32)
    weights = weights / weights.sum() * num_classes
    criterion = nn.CrossEntropyLoss(weight=weights.to(device))

    optimizer = optim.Adam(model.parameters(), lr=1e-3)
    scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

    # 7. 训练
    num_epochs = 30
    for epoch in range(num_epochs):
        model.train()
        total_loss, correct, total = 0, 0, 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            total_loss += loss.item() * X_batch.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

        train_acc = correct / total
        train_loss = total_loss / total

        # 验证
        model.eval()
        val_loss, correct, total = 0, 0, 0
        y_true, y_pred = [], []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)
                _, predicted = torch.max(outputs, 1)
                correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)
                y_true.extend(y_batch.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        val_acc = correct / total
        val_loss /= total

        scheduler.step()

        print(f"Epoch {epoch+1}/{num_epochs} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    # 8. 评估
    print("\n分类报告:")
    print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
    print("混淆矩阵:")
    print(confusion_matrix(y_true, y_pred))

    # 9. 保存模型
    torch.save(model.state_dict(), "data/lstm_bilstm_balanced_augmented.pth")
    print("✅ 模型已保存到 data/lstm_bilstm_balanced_augmented.pth")


原始 X shape: (1534, 19, 96) y shape: (1534,)
类别数: 15 类别: ['a_walk' 'p_bent' 'p_lie' 'p_sit' 'p_situp' 'p_stand' 't_bed_turn'
 't_bend' 't_lie_to_sit' 't_lie_to_situp' 't_sit_to_lie' 't_sit_to_stand'
 't_situp_to_sit' 't_stand_to_sit' 't_straighten']
平衡后 X shape: (2383, 19, 96) y shape: (2383,)
Epoch 1/30 | Train Loss: 2.5073 Acc: 0.1159 | Val Loss: 2.3586 Acc: 0.1824
Epoch 2/30 | Train Loss: 2.3422 Acc: 0.1600 | Val Loss: 2.2286 Acc: 0.1929
Epoch 3/30 | Train Loss: 2.2937 Acc: 0.1579 | Val Loss: 2.2577 Acc: 0.1803
Epoch 4/30 | Train Loss: 2.2891 Acc: 0.1710 | Val Loss: 2.2549 Acc: 0.1845
Epoch 5/30 | Train Loss: 2.2563 Acc: 0.1878 | Val Loss: 2.2271 Acc: 0.2034
Epoch 6/30 | Train Loss: 2.2614 Acc: 0.1742 | Val Loss: 2.2169 Acc: 0.1887
Epoch 7/30 | Train Loss: 2.2202 Acc: 0.1857 | Val Loss: 2.1773 Acc: 0.1908
Epoch 8/30 | Train Loss: 2.2158 Acc: 0.1910 | Val Loss: 2.1873 Acc: 0.1866
Epoch 9/30 | Train Loss: 2.1996 Acc: 0.1910 | Val Loss: 2.1831 Acc: 0.1992
Epoch 10/30 | Train Loss: 2.193

In [2]:
import numpy as np
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from collections import Counter
import random

# =====================
# 1. 固定随机种子
# =====================
def set_seed(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)

set_seed(42)

# =====================
# 2. 加载数据
# =====================
X = np.load("data/X_lstm.npy")   # shape: (N, T, F)
y = np.load("data/y_lstm.npy")   # shape: (N, )

print("原始 X shape:", X.shape, "y shape:", y.shape)

# =====================
# 3. 删除样本过少类别
# =====================
min_support = 10  # 阈值（可以改成20等）
counter = Counter(y)
print("原始类别分布:", counter)

# 筛选合法类别
valid_classes = [cls for cls, cnt in counter.items() if cnt >= min_support]
mask = np.isin(y, valid_classes)

X = X[mask]
y = y[mask]

print("过滤后 X shape:", X.shape, "y shape:", y.shape)
print("保留类别:", np.unique(y))

# =====================
# 4. 标签编码
# =====================
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
print("类别数:", num_classes, "类别:", le.classes_)

# =====================
# 5. 转换为 Tensor
# =====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
X_tensor = torch.tensor(X, dtype=torch.float32)
y_tensor = torch.tensor(y_encoded, dtype=torch.long)

# train/val split
X_train, X_val, y_train, y_val = train_test_split(
    X_tensor, y_tensor, test_size=0.2, stratify=y_tensor, random_state=42
)

train_dataset = TensorDataset(X_train, y_train)
val_dataset = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

# =====================
# 6. 定义 BiLSTM 模型
# =====================
class BiLSTMClassifier(nn.Module):
    def __init__(self, input_size, hidden_size, num_layers, num_classes, dropout=0.5):
        super().__init__()
        self.lstm = nn.LSTM(
            input_size, hidden_size, num_layers,
            batch_first=True, dropout=dropout, bidirectional=True
        )
        self.fc = nn.Linear(hidden_size * 2, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)   # (batch, seq_len, hidden*2)
        out = out[:, -1, :]     # 取最后时刻
        return self.fc(out)

# =====================
# 7. 初始化
# =====================
input_size = X.shape[2]
hidden_size = 128
num_layers = 2
model = BiLSTMClassifier(input_size, hidden_size, num_layers, num_classes).to(device)

# 类别权重（加权交叉熵）
class_counts = np.bincount(y_encoded)
weights = torch.tensor(1.0 / (class_counts + 1e-6), dtype=torch.float32).to(device)
weights = weights / weights.sum() * num_classes
criterion = nn.CrossEntropyLoss(weight=weights)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# =====================
# 8. 训练 & 验证
# =====================
def train_model(epochs=30):
    for epoch in range(epochs):
        model.train()
        train_loss, correct, total = 0, 0, 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * X_batch.size(0)
            preds = outputs.argmax(dim=1)
            correct += (preds == y_batch).sum().item()
            total += y_batch.size(0)

        scheduler.step()
        train_acc = correct / total
        train_loss /= total

        # 验证
        model.eval()
        val_loss, val_correct, val_total = 0, 0, 0
        y_true, y_pred = [], []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)
                val_loss += loss.item() * X_batch.size(0)

                preds = outputs.argmax(dim=1)
                val_correct += (preds == y_batch).sum().item()
                val_total += y_batch.size(0)

                y_true.extend(y_batch.cpu().numpy())
                y_pred.extend(preds.cpu().numpy())

        val_acc = val_correct / val_total
        val_loss /= val_total

        print(f"Epoch {epoch+1}/30 | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    # 最终报告
    print("\n分类报告:")
    print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
    print("\n混淆矩阵:")
    print(confusion_matrix(y_true, y_pred))

    torch.save(model.state_dict(), "data/lstm_bilstm_filtered.pth")
    print("✅ 模型已保存到 data/lstm_bilstm_filtered.pth")

# =====================
# 9. 开始训练
# =====================
train_model(epochs=30)


原始 X shape: (1534, 19, 96) y shape: (1534,)
原始类别分布: Counter({np.str_('p_sit'): 527, np.str_('p_stand'): 412, np.str_('a_walk'): 231, np.str_('p_lie'): 113, np.str_('p_bent'): 78, np.str_('t_bend'): 41, np.str_('t_straighten'): 35, np.str_('t_bed_turn'): 25, np.str_('t_stand_to_sit'): 18, np.str_('p_situp'): 16, np.str_('t_sit_to_stand'): 13, np.str_('t_situp_to_sit'): 10, np.str_('t_sit_to_lie'): 8, np.str_('t_lie_to_situp'): 4, np.str_('t_lie_to_sit'): 3})
过滤后 X shape: (1519, 19, 96) y shape: (1519,)
保留类别: ['a_walk' 'p_bent' 'p_lie' 'p_sit' 'p_situp' 'p_stand' 't_bed_turn'
 't_bend' 't_sit_to_stand' 't_situp_to_sit' 't_stand_to_sit'
 't_straighten']
类别数: 12 类别: ['a_walk' 'p_bent' 'p_lie' 'p_sit' 'p_situp' 'p_stand' 't_bed_turn'
 't_bend' 't_sit_to_stand' 't_situp_to_sit' 't_stand_to_sit'
 't_straighten']
Epoch 1/30 | Train Loss: 2.3980 Acc: 0.0872 | Val Loss: 2.4435 Acc: 0.1118
Epoch 2/30 | Train Loss: 2.3056 Acc: 0.1490 | Val Loss: 2.3673 Acc: 0.0757
Epoch 3/30 | Train Loss: 2.2396 A

In [5]:
import numpy as np
from collections import Counter

# 加载原始数据
X = np.load("data/X_lstm.npy")
y = np.load("data/y_lstm.npy")

print("原始 X shape:", X.shape, "y shape:", y.shape)
print("原始类别分布:", Counter(y))

# 选取前5个类别
top5 = [cls for cls, _ in Counter(y).most_common(5)]
print("保留类别:", top5)

# 过滤数据
mask = np.isin(y, top5)
X_filtered = X[mask]
y_filtered = y[mask]

print("过滤后 X shape:", X_filtered.shape, "y shape:", y_filtered.shape)
print("过滤后类别分布:", Counter(y_filtered))

# 保存过滤后的数据
np.save("X_top5.npy", X_filtered)
np.save("y_top5.npy", y_filtered)

print("✅ 已保存到 X_top5.npy / y_top5.npy")


原始 X shape: (1534, 19, 96) y shape: (1534,)
原始类别分布: Counter({np.str_('p_sit'): 527, np.str_('p_stand'): 412, np.str_('a_walk'): 231, np.str_('p_lie'): 113, np.str_('p_bent'): 78, np.str_('t_bend'): 41, np.str_('t_straighten'): 35, np.str_('t_bed_turn'): 25, np.str_('t_stand_to_sit'): 18, np.str_('p_situp'): 16, np.str_('t_sit_to_stand'): 13, np.str_('t_situp_to_sit'): 10, np.str_('t_sit_to_lie'): 8, np.str_('t_lie_to_situp'): 4, np.str_('t_lie_to_sit'): 3})
保留类别: [np.str_('p_sit'), np.str_('p_stand'), np.str_('a_walk'), np.str_('p_lie'), np.str_('p_bent')]
过滤后 X shape: (1361, 19, 96) y shape: (1361,)
过滤后类别分布: Counter({np.str_('p_sit'): 527, np.str_('p_stand'): 412, np.str_('a_walk'): 231, np.str_('p_lie'): 113, np.str_('p_bent'): 78})
✅ 已保存到 X_top5.npy / y_top5.npy


In [6]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# ====================
# 1. 数据加载
# ====================
X = np.load("X_top5.npy")   # (N, 19, 96)
y = np.load("y_top5.npy")   # (N,)

print("原始 X shape:", X.shape, "y shape:", y.shape)
print("类别分布:", Counter(y))

# 标签编码
le = LabelEncoder()
y_encoded = le.fit_transform(y)
num_classes = len(le.classes_)
print("类别数:", num_classes, "类别:", le.classes_)

# 划分训练/验证集
X_train, X_val, y_train, y_val = train_test_split(
    X, y_encoded, test_size=0.2, random_state=42, stratify=y_encoded
)

# 转换为 Tensor
X_train = torch.tensor(X_train, dtype=torch.float32)
X_val   = torch.tensor(X_val, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
y_val   = torch.tensor(y_val, dtype=torch.long)

# 构建 DataLoader
batch_size = 32
train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=batch_size, shuffle=True)
val_loader   = DataLoader(TensorDataset(X_val, y_val), batch_size=batch_size, shuffle=False)

# ====================
# 2. 定义 LSTM 模型
# ====================
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes, num_layers=1, dropout=0.3):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(
            input_size=input_dim,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,
            dropout=dropout if num_layers > 1 else 0
        )
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)   # (batch, seq_len, hidden)
        out = out[:, -1, :]     # 取最后时刻的输出
        out = self.fc(out)
        return out

# ====================
# 3. 训练准备
# ====================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(input_dim=X.shape[2], hidden_dim=128, num_classes=num_classes).to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)

# ====================
# 4. 训练循环
# ====================
def train_model(model, train_loader, val_loader, epochs=20):
    for epoch in range(epochs):
        model.train()
        train_loss, train_correct, total = 0, 0, 0
        for X_batch, y_batch in train_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)

            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_loss += loss.item() * X_batch.size(0)
            _, predicted = torch.max(outputs, 1)
            train_correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

        train_acc = train_correct / total
        train_loss /= total

        # 验证
        model.eval()
        val_loss, val_correct, total = 0, 0, 0
        y_true, y_pred = [], []
        with torch.no_grad():
            for X_batch, y_batch in val_loader:
                X_batch, y_batch = X_batch.to(device), y_batch.to(device)
                outputs = model(X_batch)
                loss = criterion(outputs, y_batch)

                val_loss += loss.item() * X_batch.size(0)
                _, predicted = torch.max(outputs, 1)
                val_correct += (predicted == y_batch).sum().item()
                total += y_batch.size(0)

                y_true.extend(y_batch.cpu().numpy())
                y_pred.extend(predicted.cpu().numpy())

        val_acc = val_correct / total
        val_loss /= total

        print(f"Epoch {epoch+1}/20 | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | "
              f"Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    return y_true, y_pred

# ====================
# 5. 运行训练
# ====================
y_true, y_pred = train_model(model, train_loader, val_loader, epochs=20)

# ====================
# 6. 评估
# ====================
print("\n分类报告:")
print(classification_report(y_true, y_pred, target_names=le.classes_))

print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))

# ====================
# 7. 保存模型
# ====================
torch.save(model.state_dict(), "data/lstm_baseline_top5.pth")
print("✅ 模型已保存到 data/lstm_baseline_top5.pth")


原始 X shape: (1361, 19, 96) y shape: (1361,)
类别分布: Counter({np.str_('p_sit'): 527, np.str_('p_stand'): 412, np.str_('a_walk'): 231, np.str_('p_lie'): 113, np.str_('p_bent'): 78})
类别数: 5 类别: ['a_walk' 'p_bent' 'p_lie' 'p_sit' 'p_stand']
Epoch 1/20 | Train Loss: 1.4543 Acc: 0.4724 | Val Loss: 1.3705 Acc: 0.4762
Epoch 2/20 | Train Loss: 1.3170 Acc: 0.5083 | Val Loss: 1.3198 Acc: 0.4725
Epoch 3/20 | Train Loss: 1.2769 Acc: 0.5156 | Val Loss: 1.2936 Acc: 0.4725
Epoch 4/20 | Train Loss: 1.2550 Acc: 0.5175 | Val Loss: 1.2843 Acc: 0.4725
Epoch 5/20 | Train Loss: 1.2490 Acc: 0.5175 | Val Loss: 1.2804 Acc: 0.4725
Epoch 6/20 | Train Loss: 1.2388 Acc: 0.5239 | Val Loss: 1.2838 Acc: 0.4762
Epoch 7/20 | Train Loss: 1.2282 Acc: 0.5276 | Val Loss: 1.2753 Acc: 0.4799
Epoch 8/20 | Train Loss: 1.2256 Acc: 0.5294 | Val Loss: 1.2768 Acc: 0.4725
Epoch 9/20 | Train Loss: 1.2204 Acc: 0.5285 | Val Loss: 1.2832 Acc: 0.4689
Epoch 10/20 | Train Loss: 1.2143 Acc: 0.5340 | Val Loss: 1.2837 Acc: 0.4689
Epoch 11/20 | 

D:\Conda\miniconda3\envs\panoptes_pc_hpe\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Conda\miniconda3\envs\panoptes_pc_hpe\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
D:\Conda\miniconda3\envs\panoptes_pc_hpe\Lib\site-packages\sklearn\metrics\_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is

In [8]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
import random

# =============== 固定随机种子 ===============
SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
torch.cuda.manual_seed_all(SEED)

# =============== 加载原始数据 ===============
X = np.load("data/X_lstm.npy")   # (N, T, F)
y = np.load("data/y_lstm.npy")   # (N,)

print("原始 X shape:", X.shape, "y shape:", y.shape)

# =============== 保留前 5 个类别 ===============
top5_classes = ['p_sit', 'p_stand', 'a_walk', 'p_lie', 'p_bent']
mask = np.isin(y, top5_classes)
X = X[mask]
y = y[mask]

print("过滤后 X shape:", X.shape, "y shape:", y.shape)
print("类别分布:", Counter(y))

# =============== 过采样函数 ===============
def balance_upsample(X, y):
    counter = Counter(y)
    max_count = max(counter.values())  # 最多类别样本数
    indices = []

    for label in counter.keys():
        label_idx = np.where(y == label)[0]
        sampled_idx = np.random.choice(label_idx, max_count, replace=True)
        indices.extend(sampled_idx)

    indices = np.array(indices)
    return X[indices], y[indices]

X_bal, y_bal = balance_upsample(X, y)
print("过采样后:", Counter(y_bal))

# =============== 标签编码 ===============
le = LabelEncoder()
y_bal = le.fit_transform(y_bal)

# 转 torch tensor
X_tensor = torch.tensor(X_bal, dtype=torch.float32)
y_tensor = torch.tensor(y_bal, dtype=torch.long)

# =============== 数据集划分 ===============
X_train, X_val, y_train, y_val = train_test_split(
    X_tensor, y_tensor, test_size=0.2, random_state=SEED, stratify=y_tensor
)

# =============== LSTM 模型定义 ===============
class LSTMClassifier(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, num_classes, dropout=0.3):
        super(LSTMClassifier, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, batch_first=True, dropout=dropout)
        self.fc = nn.Linear(hidden_dim, num_classes)

    def forward(self, x):
        out, _ = self.lstm(x)   # (batch, seq_len, hidden)
        out = out[:, -1, :]     # 取最后时刻
        out = self.fc(out)
        return out

# =============== 模型初始化 ===============
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = LSTMClassifier(input_dim=X.shape[2], hidden_dim=128, num_layers=2, num_classes=len(le.classes_))
model = model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# =============== DataLoader ===============
train_data = torch.utils.data.TensorDataset(X_train, y_train)
val_data = torch.utils.data.TensorDataset(X_val, y_val)

train_loader = torch.utils.data.DataLoader(train_data, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_data, batch_size=32, shuffle=False)

# =============== 训练循环 ===============
EPOCHS = 30
for epoch in range(EPOCHS):
    model.train()
    train_loss, correct, total = 0, 0, 0

    for X_batch, y_batch in train_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)

        optimizer.zero_grad()
        outputs = model(X_batch)
        loss = criterion(outputs, y_batch)
        loss.backward()
        optimizer.step()

        train_loss += loss.item() * X_batch.size(0)
        _, predicted = torch.max(outputs, 1)
        correct += (predicted == y_batch).sum().item()
        total += y_batch.size(0)

    train_acc = correct / total
    train_loss /= total

    # 验证
    model.eval()
    val_loss, correct, total = 0, 0, 0
    with torch.no_grad():
        for X_batch, y_batch in val_loader:
            X_batch, y_batch = X_batch.to(device), y_batch.to(device)
            outputs = model(X_batch)
            loss = criterion(outputs, y_batch)

            val_loss += loss.item() * X_batch.size(0)
            _, predicted = torch.max(outputs, 1)
            correct += (predicted == y_batch).sum().item()
            total += y_batch.size(0)

    val_acc = correct / total
    val_loss /= total

    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

# =============== 评估 ===============
y_true, y_pred = [], []
model.eval()
with torch.no_grad():
    for X_batch, y_batch in val_loader:
        X_batch, y_batch = X_batch.to(device), y_batch.to(device)
        outputs = model(X_batch)
        _, predicted = torch.max(outputs, 1)
        y_true.extend(y_batch.cpu().numpy())
        y_pred.extend(predicted.cpu().numpy())

print("\n分类报告:")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))

print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))

# =============== 保存模型 ===============
torch.save(model.state_dict(), "data/lstm_baseline_oversampled.pth")
print("✅ 模型已保存到 data/lstm_baseline_oversampled.pth")


原始 X shape: (1534, 19, 96) y shape: (1534,)
过滤后 X shape: (1361, 19, 96) y shape: (1361,)
类别分布: Counter({np.str_('p_sit'): 527, np.str_('p_stand'): 412, np.str_('a_walk'): 231, np.str_('p_lie'): 113, np.str_('p_bent'): 78})
过采样后: Counter({np.str_('a_walk'): 527, np.str_('p_stand'): 527, np.str_('p_sit'): 527, np.str_('p_lie'): 527, np.str_('p_bent'): 527})
Epoch 1/30 | Train Loss: 1.4993 Acc: 0.3401 | Val Loss: 1.4882 Acc: 0.3378
Epoch 2/30 | Train Loss: 1.4233 Acc: 0.3904 | Val Loss: 1.4716 Acc: 0.3776
Epoch 3/30 | Train Loss: 1.4122 Acc: 0.4056 | Val Loss: 1.4563 Acc: 0.3776
Epoch 4/30 | Train Loss: 1.3973 Acc: 0.4028 | Val Loss: 1.4266 Acc: 0.3909
Epoch 5/30 | Train Loss: 1.3719 Acc: 0.4151 | Val Loss: 1.4468 Acc: 0.3966
Epoch 6/30 | Train Loss: 1.3728 Acc: 0.4250 | Val Loss: 1.4127 Acc: 0.3776
Epoch 7/30 | Train Loss: 1.3778 Acc: 0.4127 | Val Loss: 1.4485 Acc: 0.3548
Epoch 8/30 | Train Loss: 1.3889 Acc: 0.3843 | Val Loss: 1.3897 Acc: 0.3852
Epoch 9/30 | Train Loss: 1.3542 Acc: 0.424

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter
import random

# ========== 1. 固定随机种子 ==========
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)

# ========== 2. 加载数据 ==========
X = np.load("data/X_lstm.npy")   # (N, T, F)
y = np.load("data/y_lstm.npy")   # (N,)
print("原始 X shape:", X.shape, "y shape:", y.shape)

# 保留前5类
keep_labels = ['p_sit', 'p_stand', 'a_walk', 'p_lie', 'p_bent']
mask = np.isin(y, keep_labels)
X, y = X[mask], y[mask]

# 编码类别
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(y)

print("过滤后 X shape:", X.shape, "y shape:", y.shape)
print("类别分布:", Counter(y))

# ========== 3. 过采样 ==========
from imblearn.over_sampling import RandomOverSampler
X_flat = X.reshape(X.shape[0], -1)
ros = RandomOverSampler(random_state=seed)
X_res, y_res = ros.fit_resample(X_flat, y)
X_res = X_res.reshape(-1, X.shape[1], X.shape[2])

print("过采样后:", Counter(y_res))

# ========== 4. 划分训练集/验证集 ==========
X_train, X_val, y_train, y_val = train_test_split(
    X_res, y_res, test_size=0.2, random_state=seed, stratify=y_res
)

# 转换为 Tensor
X_train, y_train = torch.tensor(X_train, dtype=torch.float32), torch.tensor(y_train, dtype=torch.long)
X_val, y_val = torch.tensor(X_val, dtype=torch.float32), torch.tensor(y_val, dtype=torch.long)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=32, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=32, shuffle=False)

# ========== 5. 定义模型 ==========
class AttentionLayer(nn.Module):
    def __init__(self, hidden_dim):
        super().__init__()
        self.attn = nn.Linear(hidden_dim * 2, 1)  # BiLSTM 输出 hidden*2
    def forward(self, lstm_out):
        weights = torch.softmax(self.attn(lstm_out), dim=1)   # (B, T, 1)
        context = torch.sum(weights * lstm_out, dim=1)        # (B, H*2)
        return context

class BiLSTM_Attention(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_classes):
        super().__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True, bidirectional=True)
        self.attn = AttentionLayer(hidden_dim)
        self.fc = nn.Linear(hidden_dim * 2, num_classes)
    def forward(self, x):
        out, _ = self.lstm(x)
        out = self.attn(out)  # attention pooling
        out = self.fc(out)
        return out

# ========== 6. 定义 Focal Loss ==========
class FocalLoss(nn.Module):
    def __init__(self, alpha=1, gamma=2, reduction='mean'):
        super(FocalLoss, self).__init__()
        self.alpha = alpha
        self.gamma = gamma
        self.reduction = reduction
    def forward(self, inputs, targets):
        ce_loss = nn.CrossEntropyLoss(reduction='none')(inputs, targets)
        pt = torch.exp(-ce_loss)
        loss = self.alpha * (1-pt)**self.gamma * ce_loss
        if self.reduction == 'mean':
            return loss.mean()
        else:
            return loss.sum()

# ========== 7. 训练 ==========
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = BiLSTM_Attention(input_dim=X.shape[2], hidden_dim=64, num_classes=len(le.classes_)).to(device)
criterion = FocalLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3)

EPOCHS = 30
for epoch in range(EPOCHS):
    # ---- train ----
    model.train()
    train_loss, correct, total = 0, 0, 0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(Xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * Xb.size(0)
        correct += (outputs.argmax(1) == yb).sum().item()
        total += yb.size(0)
    train_acc = correct / total
    train_loss /= total

    # ---- val ----
    model.eval()
    val_loss, correct, total = 0, 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            outputs = model(Xb)
            loss = criterion(outputs, yb)
            val_loss += loss.item() * Xb.size(0)
            correct += (outputs.argmax(1) == yb).sum().item()
            total += yb.size(0)
            y_true.extend(yb.cpu().numpy())
            y_pred.extend(outputs.argmax(1).cpu().numpy())
    val_acc = correct / total
    val_loss /= total

    scheduler.step(val_loss)
    print(f"Epoch {epoch+1}/{EPOCHS} | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

# ========== 8. 评估 ==========
print("\n分类报告:")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))

torch.save(model.state_dict(), "data/lstm_bilstm_attention_focal.pth")
print("✅ 模型已保存到 data/lstm_bilstm_attention_focal.pth")


原始 X shape: (1534, 19, 96) y shape: (1534,)
过滤后 X shape: (1361, 19, 96) y shape: (1361,)
类别分布: Counter({np.int64(3): 527, np.int64(4): 412, np.int64(0): 231, np.int64(2): 113, np.int64(1): 78})
过采样后: Counter({np.int64(0): 527, np.int64(4): 527, np.int64(3): 527, np.int64(2): 527, np.int64(1): 527})
Epoch 1/30 | Train Loss: 0.9401 Acc: 0.3335 | Val Loss: 0.9204 Acc: 0.3321
Epoch 2/30 | Train Loss: 0.8667 Acc: 0.3871 | Val Loss: 0.9037 Acc: 0.3510
Epoch 3/30 | Train Loss: 0.8531 Acc: 0.4004 | Val Loss: 0.8886 Acc: 0.3890
Epoch 4/30 | Train Loss: 0.8369 Acc: 0.4151 | Val Loss: 0.8751 Acc: 0.3757
Epoch 5/30 | Train Loss: 0.8317 Acc: 0.4213 | Val Loss: 0.8735 Acc: 0.4099
Epoch 6/30 | Train Loss: 0.8375 Acc: 0.4156 | Val Loss: 0.8864 Acc: 0.3757
Epoch 7/30 | Train Loss: 0.8241 Acc: 0.4141 | Val Loss: 0.8758 Acc: 0.3947
Epoch 8/30 | Train Loss: 0.8150 Acc: 0.4274 | Val Loss: 0.8768 Acc: 0.3985
Epoch 9/30 | Train Loss: 0.8118 Acc: 0.4326 | Val Loss: 0.8590 Acc: 0.3890
Epoch 10/30 | Train Loss:

In [10]:
pip install imbalanced-learn


Note: you may need to restart the kernel to use updated packages.


In [3]:
import os
import numpy as np
from collections import Counter

# ============ 1. 数据准备 ============
# 这里替换成你的原始 X, y 数据来源
# 假设 X.shape = (1534, 19, 96), y.shape = (1534,)
# 如果已经有 X, y，就不用再生成，直接用
# X, y = ...  # 你之前的预处理代码

# 文件名
X_file = "X_train.npy"
y_file = "y_train.npy"

# 如果文件不存在，就保存
if not os.path.exists(X_file) or not os.path.exists(y_file):
    print("⚠️ 没找到缓存文件，正在保存...")
    np.save(X_file, X)
    np.save(y_file, y)

# 加载
X = np.load(X_file)
y = np.load(y_file)

print("数据 shape:", X.shape, y.shape, 
      "类别数:", len(np.unique(y)), 
      "类别分布:", Counter(y))


⚠️ 没找到缓存文件，正在保存...
数据 shape: (1361, 19, 96) (1361,) 类别数: 5 类别分布: Counter({np.int64(3): 527, np.int64(4): 412, np.int64(0): 231, np.int64(2): 113, np.int64(1): 78})


In [18]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset, random_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import train_test_split
from sklearn.utils import resample
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

# ============ 1. 数据准备 ============
seed = 42
torch.manual_seed(seed)
np.random.seed(seed)

# 假设数据在 data 文件夹
X = np.load("data/X.npy")   # (N, T, F) 或 (N, 100, 32, 3)
y = np.load("data/y.npy")   # (N,)

print("原始 X shape:", X.shape, "y shape:", y.shape)
print("原始类别分布:", Counter(y))

# 如果 X 是 4D (N, T, J, C)，展开为 (N, T, J*C)
if X.ndim == 4:
    N, T, J, C = X.shape
    X = X.reshape(N, T, J*C)

# 只保留前 5 个类别
top5 = ['p_sit', 'p_stand', 'a_walk', 'p_lie', 'p_bent']
label_map = {cls: i for i, cls in enumerate(top5)}

mask = np.isin(y, top5)
X = X[mask]
y = y[mask]

# 转换为数字编码
y = np.array([label_map[label] for label in y])

print("过滤后 X shape:", X.shape, "y shape:", y.shape)
print("过滤后类别分布:", Counter(y))

# ============ 2. 过采样 ============
X_flat = X.reshape(X.shape[0], -1)
ros = RandomOverSampler(random_state=seed)
X_res, y_res = ros.fit_resample(X_flat, y)
X_res = X_res.reshape(X_res.shape[0], X.shape[1], X.shape[2])

print("过采样后 X shape:", X_res.shape, "y shape:", y_res.shape)
print("过采样后类别分布:", Counter(y_res))

# ============ 3. 数据划分 ============
X_train, X_val, y_train, y_val = train_test_split(
    X_res, y_res, test_size=0.2, random_state=seed, stratify=y_res
)

X_train = torch.tensor(X_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.long)
X_val = torch.tensor(X_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.long)

train_ds = TensorDataset(X_train, y_train)
val_ds = TensorDataset(X_val, y_val)

train_loader = DataLoader(train_ds, batch_size=32, shuffle=True)
val_loader = DataLoader(val_ds, batch_size=32, shuffle=False)

# ============ 4. Transformer 模型 ============
class TransformerClassifier(nn.Module):
    def __init__(self, input_dim, num_classes, num_heads=4, num_layers=2, hidden_dim=128):
        super().__init__()
        encoder_layer = nn.TransformerEncoderLayer(d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim)
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        # x: (B, T, F) -> (T, B, F)
        x = x.permute(1, 0, 2)
        out = self.transformer(x)  # (T, B, F)
        out = out.mean(dim=0)      # 池化 (B, F)
        return self.fc(out)

model = TransformerClassifier(input_dim=X.shape[2], num_classes=len(top5))
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# ============ 5. 训练配置 ============
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.7)

# EarlyStopping
class EarlyStopping:
    def __init__(self, patience=5, delta=0):
        self.patience = patience
        self.delta = delta
        self.best_loss = None
        self.counter = 0
        self.early_stop = False

    def __call__(self, val_loss):
        if self.best_loss is None or val_loss < self.best_loss - self.delta:
            self.best_loss = val_loss
            self.counter = 0
        else:
            self.counter += 1
            if self.counter >= self.patience:
                self.early_stop = True

early_stopping = EarlyStopping(patience=5)

# ============ 6. 训练循环 ============
for epoch in range(30):
    # ---- Train ----
    model.train()
    train_loss, correct, total = 0, 0, 0
    for Xb, yb in train_loader:
        Xb, yb = Xb.to(device), yb.to(device)
        optimizer.zero_grad()
        outputs = model(Xb)
        loss = criterion(outputs, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * yb.size(0)
        _, preds = torch.max(outputs, 1)
        correct += (preds == yb).sum().item()
        total += yb.size(0)

    train_acc = correct / total
    train_loss /= total

    # ---- Validation ----
    model.eval()
    val_loss, correct, total = 0, 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for Xb, yb in val_loader:
            Xb, yb = Xb.to(device), yb.to(device)
            outputs = model(Xb)
            loss = criterion(outputs, yb)
            val_loss += loss.item() * yb.size(0)
            _, preds = torch.max(outputs, 1)
            correct += (preds == yb).sum().item()
            total += yb.size(0)
            y_true.extend(yb.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    val_acc = correct / total
    val_loss /= total
    print(f"Epoch {epoch+1}/30 | Train Loss: {train_loss:.4f} Acc: {train_acc:.4f} | Val Loss: {val_loss:.4f} Acc: {val_acc:.4f}")

    scheduler.step()
    early_stopping(val_loss)
    if early_stopping.early_stop:
        print("⏹ Early stopping triggered")
        break

# ============ 7. 评估 ============
print("\n分类报告:")
print(classification_report(y_true, y_pred, target_names=top5, zero_division=0))

print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))

torch.save(model.state_dict(), "data/transformer_top5.pth")
print("✅ 模型已保存到 data/transformer_top5.pth")


原始 X shape: (144, 100, 32, 3) y shape: (144,)
原始类别分布: Counter({np.str_('p_stand'): 45, np.str_('a_walk'): 28, np.str_('p_sit'): 27, np.str_('Unknown'): 17, np.str_('t_bend'): 7, np.str_('p_lie'): 6, np.str_('p_bent'): 6, np.str_('t_straighten'): 4, np.str_('t_bed_turn'): 3, np.str_('t_stand_to_sit'): 1})
过滤后 X shape: (112, 100, 96) y shape: (112,)
过滤后类别分布: Counter({np.int64(1): 45, np.int64(2): 28, np.int64(0): 27, np.int64(3): 6, np.int64(4): 6})
过采样后 X shape: (225, 100, 96) y shape: (225,)
过采样后类别分布: Counter({np.int64(2): 45, np.int64(1): 45, np.int64(0): 45, np.int64(3): 45, np.int64(4): 45})


D:\Conda\miniconda3\envs\panoptes_pc_hpe\Lib\site-packages\torch\nn\modules\transformer.py:379: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


Epoch 1/30 | Train Loss: 1.5930 Acc: 0.2444 | Val Loss: 1.5683 Acc: 0.2222
Epoch 2/30 | Train Loss: 1.4679 Acc: 0.4056 | Val Loss: 1.4524 Acc: 0.3778
Epoch 3/30 | Train Loss: 1.3278 Acc: 0.4944 | Val Loss: 1.4099 Acc: 0.4667
Epoch 4/30 | Train Loss: 1.2309 Acc: 0.5556 | Val Loss: 1.4214 Acc: 0.4444
Epoch 5/30 | Train Loss: 1.1778 Acc: 0.5389 | Val Loss: 1.3363 Acc: 0.4444
Epoch 6/30 | Train Loss: 1.0839 Acc: 0.6111 | Val Loss: 1.2882 Acc: 0.4889
Epoch 7/30 | Train Loss: 1.0953 Acc: 0.6000 | Val Loss: 1.2049 Acc: 0.5111
Epoch 8/30 | Train Loss: 0.9861 Acc: 0.6667 | Val Loss: 1.1126 Acc: 0.6222
Epoch 9/30 | Train Loss: 0.8936 Acc: 0.6722 | Val Loss: 1.0200 Acc: 0.5778
Epoch 10/30 | Train Loss: 0.8583 Acc: 0.7333 | Val Loss: 1.0751 Acc: 0.6222
Epoch 11/30 | Train Loss: 0.7878 Acc: 0.7111 | Val Loss: 0.9554 Acc: 0.6667
Epoch 12/30 | Train Loss: 0.7027 Acc: 0.7500 | Val Loss: 1.0097 Acc: 0.6222
Epoch 13/30 | Train Loss: 0.6835 Acc: 0.7500 | Val Loss: 0.9873 Acc: 0.6000
Epoch 14/30 | Train L

In [3]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

# ===========================================================
# Step 1. 自动检测并加载数据
# ===========================================================
if os.path.exists("data/pose_data.npy") and os.path.exists("data/pose_labels.npy"):
    data_path, label_path = "data/pose_data.npy", "data/pose_labels.npy"
elif os.path.exists("data/X.npy") and os.path.exists("data/y.npy"):
    data_path, label_path = "data/X.npy", "data/y.npy"
else:
    print("⚠️ 没找到数据文件！请确认 data/ 下存在 pose_data.npy 或 X.npy")
    print("当前目录内容：", os.listdir("data"))
    sys.exit(1)

data = np.load(data_path, allow_pickle=True)
labels = np.load(label_path, allow_pickle=True)
X = np.array(data, dtype=np.float32)
y = np.array(labels)

print(f"原始 X shape: {X.shape} y shape: {y.shape}")
print("原始类别分布:", Counter(y))

# ===========================================================
# Step 2. 改进版坏样本过滤函数（带日志输出）
# ===========================================================
def filter_invalid_samples(X, y, max_abs=10000.0, min_std=1e-6, log_path="data/bad_samples.txt"):
    valid_idx, bad_info = [], []
    for i in range(len(X)):
        xi = X[i]
        reason = None
        if np.isnan(xi).any() or np.isinf(xi).any():
            reason = "NaN/Inf"
        elif np.allclose(xi, 0):
            reason = "All zeros"
        elif np.std(xi) < min_std:
            reason = f"Low std ({np.std(xi):.2e})"
        elif np.any(np.abs(xi) > max_abs):
            reason = f"Out of range (max={np.max(np.abs(xi)):.1f})"

        if reason:
            bad_info.append(f"Sample {i}: {reason}")
        else:
            valid_idx.append(i)

    with open(log_path, "w", encoding="utf-8") as f:
        if bad_info:
            f.write("\n".join(bad_info))
        else:
            f.write("✅ 无坏样本\n")

    print(f"检测并移除 {len(bad_info)} 个坏样本（剩余 {len(valid_idx)} 个），日志已保存到 {log_path}")
    return X[valid_idx], y[valid_idx]

X, y = filter_invalid_samples(X, y)
print(f"过滤后 X shape: {X.shape} y shape: {y.shape}")

# ===========================================================
# Step 3. 只保留前5类
# ===========================================================
top5 = [cls for cls, _ in Counter(y).most_common(5)]
mask = np.isin(y, top5)
X, y = X[mask], y[mask]
print(f"过滤后 X shape: {X.shape} y shape: {y.shape}")
print("保留类别:", top5)

# 标签编码（防止类别是字符串）
le = LabelEncoder()
y = le.fit_transform(y)
print(f"类别数: {len(le.classes_)} 类别映射:", dict(zip(le.classes_, range(len(le.classes_)))))

# ===========================================================
# Step 4. 展平 + 过采样
# ===========================================================
if X.ndim == 4:
    N, T, J, C = X.shape
    X = X.reshape(N, T, J * C)
elif X.ndim == 3:
    N, T, F = X.shape
    J, C = F, 1
else:
    raise ValueError(f"❌ 输入维度异常: {X.shape}")

ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X.reshape(N, -1), y)
X_res = X_res.reshape(-1, T, J * C)
print(f"过采样后 X shape: {X_res.shape} y shape: {y_res.shape}")
print("过采样后类别分布:", Counter(y_res))

# ===========================================================
# Step 5. 数据划分与Tensor化
# ===========================================================
X_train, X_val, y_train, y_val = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)
X_train, X_val = map(torch.tensor, (X_train, X_val))
y_train, y_val = map(torch.tensor, (y_train, y_val))

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=16, shuffle=False)

print(f"Train: {X_train.shape} Val: {X_val.shape}")

# ===========================================================
# Step 6. Transformer 模型定义
# ===========================================================
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=128, nhead=4, num_layers=2, dim_feedforward=256, dropout=0.1):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer(x)
        x = self.norm(x)
        x = x.mean(dim=1)
        x = self.dropout(x)
        return self.fc(x)

# ===========================================================
# Step 7. 训练配置
# ===========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(input_dim=X_train.shape[2], num_classes=len(le.classes_)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# ===========================================================
# Step 8. 训练与验证
# ===========================================================
num_epochs = 40
best_val_acc = 0

for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss, train_correct = 0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
        train_correct += (out.argmax(1) == yb).sum().item()

    train_acc = train_correct / len(train_loader.dataset)

    model.eval()
    val_loss, val_correct = 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            val_loss += loss.item() * xb.size(0)
            preds = out.argmax(1)
            val_correct += (preds == yb).sum().item()
            y_true.extend(yb.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    val_acc = val_correct / len(val_loader.dataset)
    scheduler.step()

    print(f"Epoch {epoch:02d}/{num_epochs} | Train Loss: {train_loss/len(train_loader.dataset):.4f} "
          f"Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader.dataset):.4f} "
          f"Acc: {val_acc:.4f} | LR: {scheduler.get_last_lr()}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "data/transformer_top5_filtered.pth")

# ===========================================================
# Step 9. 输出报告
# ===========================================================
print(f"\n✅ 最佳验证准确率: {best_val_acc:.4f}")
print("模型已保存到 data/transformer_top5_filtered.pth")

print("\n分类报告:")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))


原始 X shape: (144, 100, 32, 3) y shape: (144,)
原始类别分布: Counter({np.str_('p_stand'): 45, np.str_('a_walk'): 28, np.str_('p_sit'): 27, np.str_('Unknown'): 17, np.str_('t_bend'): 7, np.str_('p_lie'): 6, np.str_('p_bent'): 6, np.str_('t_straighten'): 4, np.str_('t_bed_turn'): 3, np.str_('t_stand_to_sit'): 1})
检测并移除 0 个坏样本（剩余 144 个），日志已保存到 data/bad_samples.txt
过滤后 X shape: (144, 100, 32, 3) y shape: (144,)
过滤后 X shape: (124, 100, 32, 3) y shape: (124,)
保留类别: [np.str_('p_stand'), np.str_('a_walk'), np.str_('p_sit'), np.str_('Unknown'), np.str_('t_bend')]
类别数: 5 类别映射: {np.str_('Unknown'): 0, np.str_('a_walk'): 1, np.str_('p_sit'): 2, np.str_('p_stand'): 3, np.str_('t_bend'): 4}
过采样后 X shape: (225, 100, 96) y shape: (225,)
过采样后类别分布: Counter({np.int64(1): 45, np.int64(3): 45, np.int64(2): 45, np.int64(0): 45, np.int64(4): 45})
Train: torch.Size([180, 100, 96]) Val: torch.Size([45, 100, 96])
Epoch 01/40 | Train Loss: 1.7452 Acc: 0.2278 | Val Loss: 1.6312 Acc: 0.2889 | LR: [0.001]
Epoch 02/40 | Tr

In [4]:
import os
import sys
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.preprocessing import LabelEncoder
from collections import Counter
from imblearn.over_sampling import RandomOverSampler

# ===========================================================
# Step 1. 自动检测并加载数据
# ===========================================================
if os.path.exists("data/pose_data.npy") and os.path.exists("data/pose_labels.npy"):
    data_path, label_path = "data/pose_data.npy", "data/pose_labels.npy"
elif os.path.exists("data/X.npy") and os.path.exists("data/y.npy"):
    data_path, label_path = "data/X.npy", "data/y.npy"
else:
    print("⚠️ 没找到数据文件！请确认 data/ 下存在 pose_data.npy 或 X.npy")
    print("当前目录内容：", os.listdir("data"))
    sys.exit(1)

data = np.load(data_path, allow_pickle=True)
labels = np.load(label_path, allow_pickle=True)
X = np.array(data, dtype=np.float32)
y = np.array(labels)

print(f"原始 X shape: {X.shape} y shape: {y.shape}")
print("原始类别分布:", Counter(y))

# ===========================================================
# Step 2. 改进版坏样本过滤（带日志）
# ===========================================================
def filter_invalid_samples(X, y, max_abs=10000.0, min_std=1e-6, log_path="data/bad_samples.txt"):
    valid_idx, bad_info = [], []
    for i in range(len(X)):
        xi = X[i]
        reason = None
        if np.isnan(xi).any() or np.isinf(xi).any():
            reason = "NaN/Inf"
        elif np.allclose(xi, 0):
            reason = "All zeros"
        elif np.std(xi) < min_std:
            reason = f"Low std ({np.std(xi):.2e})"
        elif np.any(np.abs(xi) > max_abs):
            reason = f"Out of range (max={np.max(np.abs(xi)):.1f})"

        if reason:
            bad_info.append(f"Sample {i}: {reason}")
        else:
            valid_idx.append(i)

    with open(log_path, "w", encoding="utf-8") as f:
        if bad_info:
            f.write("\n".join(bad_info))
        else:
            f.write("✅ 无坏样本\n")

    print(f"检测并移除 {len(bad_info)} 个坏样本（剩余 {len(valid_idx)} 个），日志已保存到 {log_path}")
    return X[valid_idx], y[valid_idx]

X, y = filter_invalid_samples(X, y)
print(f"过滤后 X shape: {X.shape} y shape: {y.shape}")

# ===========================================================
# Step 3. 保留指定的五个类别
# ===========================================================
target_classes = ['p_sit', 'p_stand', 'a_walk', 'p_lie', 't_bend']
mask = np.isin(y, target_classes)
X, y = X[mask], y[mask]
print(f"保留类别: {target_classes}")
print(f"过滤后 X shape: {X.shape} y shape: {y.shape}")
print("过滤后类别分布:", Counter(y))

# 标签编码
le = LabelEncoder()
y = le.fit_transform(y)
print(f"类别映射: {dict(zip(le.classes_, range(len(le.classes_))))}")

# ===========================================================
# Step 4. 展平 + 过采样
# ===========================================================
if X.ndim == 4:
    N, T, J, C = X.shape
    X = X.reshape(N, T, J * C)
elif X.ndim == 3:
    N, T, F = X.shape
    J, C = F, 1
else:
    raise ValueError(f"❌ 输入维度异常: {X.shape}")

ros = RandomOverSampler(random_state=42)
X_res, y_res = ros.fit_resample(X.reshape(N, -1), y)
X_res = X_res.reshape(-1, T, J * C)
print(f"过采样后 X shape: {X_res.shape} y shape: {y_res.shape}")
print("过采样后类别分布:", Counter(y_res))

# ===========================================================
# Step 5. 数据划分
# ===========================================================
X_train, X_val, y_train, y_val = train_test_split(
    X_res, y_res, test_size=0.2, random_state=42, stratify=y_res
)
X_train, X_val = map(torch.tensor, (X_train, X_val))
y_train, y_val = map(torch.tensor, (y_train, y_val))

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=16, shuffle=False)

print(f"Train: {X_train.shape} Val: {X_val.shape}")

# ===========================================================
# Step 6. Transformer 模型
# ===========================================================
class TransformerModel(nn.Module):
    def __init__(self, input_dim, num_classes, d_model=128, nhead=4, num_layers=2, dim_feedforward=256, dropout=0.1):
        super().__init__()
        self.input_fc = nn.Linear(input_dim, d_model)
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=d_model, nhead=nhead, dim_feedforward=dim_feedforward, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.fc = nn.Linear(d_model, num_classes)
        self.dropout = nn.Dropout(dropout)
        self.norm = nn.LayerNorm(d_model)

    def forward(self, x):
        x = self.input_fc(x)
        x = self.transformer(x)
        x = self.norm(x)
        x = x.mean(dim=1)
        x = self.dropout(x)
        return self.fc(x)

# ===========================================================
# Step 7. 训练配置
# ===========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = TransformerModel(input_dim=X_train.shape[2], num_classes=len(le.classes_)).to(device)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
criterion = nn.CrossEntropyLoss()
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# ===========================================================
# Step 8. 训练与验证
# ===========================================================
num_epochs = 40
best_val_acc = 0

for epoch in range(1, num_epochs + 1):
    model.train()
    train_loss, train_correct = 0, 0

    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
        train_correct += (out.argmax(1) == yb).sum().item()

    train_acc = train_correct / len(train_loader.dataset)

    model.eval()
    val_loss, val_correct = 0, 0
    y_true, y_pred = [], []
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            val_loss += loss.item() * xb.size(0)
            preds = out.argmax(1)
            val_correct += (preds == yb).sum().item()
            y_true.extend(yb.cpu().numpy())
            y_pred.extend(preds.cpu().numpy())

    val_acc = val_correct / len(val_loader.dataset)
    scheduler.step()

    print(f"Epoch {epoch:02d}/{num_epochs} | Train Loss: {train_loss/len(train_loader.dataset):.4f} "
          f"Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader.dataset):.4f} "
          f"Acc: {val_acc:.4f} | LR: {scheduler.get_last_lr()}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "data/transformer_pose5.pth")

# ===========================================================
# Step 9. 输出报告
# ===========================================================
print(f"\n✅ 最佳验证准确率: {best_val_acc:.4f}")
print("模型已保存到 data/transformer_pose5.pth")

print("\n分类报告:")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))


原始 X shape: (144, 100, 32, 3) y shape: (144,)
原始类别分布: Counter({np.str_('p_stand'): 45, np.str_('a_walk'): 28, np.str_('p_sit'): 27, np.str_('Unknown'): 17, np.str_('t_bend'): 7, np.str_('p_lie'): 6, np.str_('p_bent'): 6, np.str_('t_straighten'): 4, np.str_('t_bed_turn'): 3, np.str_('t_stand_to_sit'): 1})
检测并移除 0 个坏样本（剩余 144 个），日志已保存到 data/bad_samples.txt
过滤后 X shape: (144, 100, 32, 3) y shape: (144,)
保留类别: ['p_sit', 'p_stand', 'a_walk', 'p_lie', 't_bend']
过滤后 X shape: (113, 100, 32, 3) y shape: (113,)
过滤后类别分布: Counter({np.str_('p_stand'): 45, np.str_('a_walk'): 28, np.str_('p_sit'): 27, np.str_('t_bend'): 7, np.str_('p_lie'): 6})
类别映射: {np.str_('a_walk'): 0, np.str_('p_lie'): 1, np.str_('p_sit'): 2, np.str_('p_stand'): 3, np.str_('t_bend'): 4}
过采样后 X shape: (225, 100, 96) y shape: (225,)
过采样后类别分布: Counter({np.int64(0): 45, np.int64(3): 45, np.int64(2): 45, np.int64(1): 45, np.int64(4): 45})
Train: torch.Size([180, 100, 96]) Val: torch.Size([45, 100, 96])
Epoch 01/40 | Train Loss: 1.686

In [1]:
import os, numpy as np, torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from collections import Counter

# ===========================================================
# Step 1. 加载数据
# ===========================================================
data_path = "data"
X = np.load(os.path.join(data_path, "X.npy"))   # (N, frames, joints, 3)
y = np.load(os.path.join(data_path, "y.npy"))

print(f"原始 X shape: {X.shape} y shape: {y.shape}")
print("原始类别分布:", Counter(y))

# ===========================================================
# Step 2. 过滤坏样本
# ===========================================================
def filter_invalid_samples(X, y, max_abs=10000.0, min_std=1e-6, log_path="data/bad_samples.txt"):
    valid_idx, bad_info = [], []
    for i in range(len(X)):
        xi = X[i]
        reason = None
        if np.isnan(xi).any() or np.isinf(xi).any():
            reason = "NaN/Inf"
        elif np.allclose(xi, 0):
            reason = "All zeros"
        elif np.std(xi) < min_std:
            reason = f"Low std ({np.std(xi):.2e})"
        elif np.any(np.abs(xi) > max_abs):
            reason = f"Out of range (max={np.max(np.abs(xi)):.1f})"

        if reason:
            bad_info.append(f"Sample {i}: {reason}")
        else:
            valid_idx.append(i)

    with open(log_path, "w", encoding="utf-8") as f:
        if bad_info:
            f.write("\n".join(bad_info))
        else:
            f.write("✅ 无坏样本\n")

    print(f"检测并移除 {len(bad_info)} 个坏样本（剩余 {len(valid_idx)} 个），日志已保存到 {log_path}")
    return X[valid_idx], y[valid_idx]

X, y = filter_invalid_samples(X, y)

# ===========================================================
# Step 3. 筛选目标类别
# ===========================================================
target_labels = ["p_sit", "p_stand", "a_walk", "p_lie", "t_bend"]
mask = np.isin(y, target_labels)
X, y = X[mask], y[mask]

print(f"过滤后 X shape: {X.shape} y shape: {y.shape}")
print("类别数:", len(set(y)))

# ===========================================================
# Step 4. 转换为 (frames, features)
# ===========================================================
N, T, J, C = X.shape
X = X.reshape(N, T, J * C)
print(f"特征展开后: {X.shape}")

# ===========================================================
# Step 5. 标签编码 + 过采样
# ===========================================================
le = LabelEncoder()
y = le.fit_transform(y)
print("类别映射:", dict(zip(le.classes_, range(len(le.classes_)))))

# 简单过采样
from sklearn.utils import resample
X_bal, y_bal = [], []
for cls in np.unique(y):
    X_cls = X[y == cls]
    y_cls = y[y == cls]
    X_up, y_up = resample(X_cls, y_cls, replace=True, n_samples=max(Counter(y).values()), random_state=42)
    X_bal.append(X_up)
    y_bal.append(y_up)
X_res = np.vstack(X_bal)
y_res = np.hstack(y_bal)
print("过采样后类别分布:", Counter(y_res))

# ===========================================================
# Step 6. 数据划分 (Train/Val/Test)
# ===========================================================
X_temp, X_test, y_temp, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)

print(f"Train: {X_train.shape} Val: {X_val.shape} Test: {X_test.shape}")

# 转为 Tensor
X_train, X_val, X_test = map(lambda x: torch.tensor(x, dtype=torch.float32), [X_train, X_val, X_test])
y_train, y_val, y_test = map(lambda y: torch.tensor(y, dtype=torch.long), [y_train, y_val, y_test])

# ===========================================================
# Step 7. Transformer 模型定义
# ===========================================================
class PoseTransformer(nn.Module):
    def __init__(self, input_dim=96, num_classes=5, num_heads=4, num_layers=2, hidden_dim=128, dropout=0.2):
        super().__init__()
        self.pos_encoder = nn.Parameter(torch.randn(1, 100, input_dim))  # learnable positional encoding
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(input_dim)
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = x + self.pos_encoder[:, :x.size(1), :]
        out = self.transformer(x)
        out = self.norm(out[:, -1, :])  # 取最后一帧
        return self.fc(out)

# ===========================================================
# Step 8. 训练流程
# ===========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PoseTransformer(input_dim=X_train.shape[2], num_classes=len(le.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=16, shuffle=False)

best_acc, patience, wait = 0.0, 8, 0
for epoch in range(1, 41):
    model.train()
    train_loss, train_correct = 0, 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
        train_correct += (out.argmax(1) == yb).sum().item()

    scheduler.step()
    train_acc = train_correct / len(train_loader.dataset)

    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            val_loss += loss.item() * xb.size(0)
            val_correct += (out.argmax(1) == yb).sum().item()

    val_acc = val_correct / len(val_loader.dataset)
    print(f"Epoch {epoch:02d}/40 | Train Loss: {train_loss/len(train_loader.dataset):.4f} "
          f"Acc: {train_acc:.4f} | Val Loss: {val_loss/len(val_loader.dataset):.4f} "
          f"Acc: {val_acc:.4f} | LR: {scheduler.get_last_lr()}")

    if val_acc > best_acc:
        best_acc, wait = val_acc, 0
        torch.save(model.state_dict(), "data/transformer_pose5.pth")
    else:
        wait += 1
        if wait >= patience:
            print("⚠️ 提前停止：验证集不再提升。")
            break

print(f"✅ 最佳验证准确率: {best_acc:.4f}")

# ===========================================================
# Step 9. Test 阶段
# ===========================================================
model.load_state_dict(torch.load("data/transformer_pose5.pth"))
model.eval()

test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=16, shuffle=False)
y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        preds = out.argmax(1)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n🎯 最终测试集性能 (Test Set):")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))


原始 X shape: (144, 100, 32, 3) y shape: (144,)
原始类别分布: Counter({np.str_('p_stand'): 45, np.str_('a_walk'): 28, np.str_('p_sit'): 27, np.str_('Unknown'): 17, np.str_('t_bend'): 7, np.str_('p_lie'): 6, np.str_('p_bent'): 6, np.str_('t_straighten'): 4, np.str_('t_bed_turn'): 3, np.str_('t_stand_to_sit'): 1})
检测并移除 0 个坏样本（剩余 144 个），日志已保存到 data/bad_samples.txt
过滤后 X shape: (113, 100, 32, 3) y shape: (113,)
类别数: 5
特征展开后: (113, 100, 96)
类别映射: {np.str_('a_walk'): 0, np.str_('p_lie'): 1, np.str_('p_sit'): 2, np.str_('p_stand'): 3, np.str_('t_bend'): 4}
过采样后类别分布: Counter({np.int64(0): 45, np.int64(1): 45, np.int64(2): 45, np.int64(3): 45, np.int64(4): 45})
Train: (135, 100, 96) Val: (45, 100, 96) Test: (45, 100, 96)
Epoch 01/40 | Train Loss: 1.7134 Acc: 0.1778 | Val Loss: 1.5140 Acc: 0.5111 | LR: [0.001]
Epoch 02/40 | Train Loss: 1.5441 Acc: 0.3037 | Val Loss: 1.3954 Acc: 0.4222 | LR: [0.001]
Epoch 03/40 | Train Loss: 1.4574 Acc: 0.3852 | Val Loss: 1.2542 Acc: 0.4667 | LR: [0.001]
Epoch 04/40 | T

C:\Users\LENOVO\AppData\Local\Temp\ipykernel_12508\3346585549.py:179: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load("data/transformer_pose5.

In [2]:
import os, numpy as np, torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.utils import resample
from collections import Counter

# ===========================================================
# Step 1. 数据加载
# ===========================================================
data_path = "data"
X = np.load(os.path.join(data_path, "X.npy"))
y = np.load(os.path.join(data_path, "y.npy"))

print(f"原始 X shape: {X.shape} y shape: {y.shape}")
print("原始类别分布:", Counter(y))

# ===========================================================
# Step 2. 过滤坏样本
# ===========================================================
def filter_invalid_samples(X, y, max_abs=10000.0, min_std=1e-6, log_path="data/bad_samples.txt"):
    valid_idx, bad_info = [], []
    for i in range(len(X)):
        xi = X[i]
        reason = None
        if np.isnan(xi).any() or np.isinf(xi).any():
            reason = "NaN/Inf"
        elif np.allclose(xi, 0):
            reason = "All zeros"
        elif np.std(xi) < min_std:
            reason = f"Low std ({np.std(xi):.2e})"
        elif np.any(np.abs(xi) > max_abs):
            reason = f"Out of range (max={np.max(np.abs(xi)):.1f})"

        if reason:
            bad_info.append(f"Sample {i}: {reason}")
        else:
            valid_idx.append(i)

    with open(log_path, "w", encoding="utf-8") as f:
        if bad_info:
            f.write("\n".join(bad_info))
        else:
            f.write("✅ 无坏样本\n")

    print(f"检测并移除 {len(bad_info)} 个坏样本（剩余 {len(valid_idx)} 个），日志已保存到 {log_path}")
    return X[valid_idx], y[valid_idx]

X, y = filter_invalid_samples(X, y)

# ===========================================================
# Step 3. 筛选目标类别
# ===========================================================
target_labels = ["p_sit", "p_stand", "a_walk", "p_lie", "t_bend"]
mask = np.isin(y, target_labels)
X, y = X[mask], y[mask]

print(f"过滤后 X shape: {X.shape} y shape: {y.shape}")

# ===========================================================
# Step 4. 特征展开
# ===========================================================
N, T, J, C = X.shape
X = X.reshape(N, T, J * C)
print(f"特征展开后: {X.shape}")

# ===========================================================
# Step 5. 标签编码 + 过采样
# ===========================================================
le = LabelEncoder()
y = le.fit_transform(y)
print("类别映射:", dict(zip(le.classes_, range(len(le.classes_)))))

X_bal, y_bal = [], []
for cls in np.unique(y):
    X_cls = X[y == cls]
    y_cls = y[y == cls]
    X_up, y_up = resample(X_cls, y_cls, replace=True, n_samples=max(Counter(y).values()), random_state=42)
    X_bal.append(X_up)
    y_bal.append(y_up)
X_res = np.vstack(X_bal)
y_res = np.hstack(y_bal)
print("过采样后类别分布:", Counter(y_res))

# ===========================================================
# Step 6. 数据划分 (Train / Val / Test)
# ===========================================================
X_temp, X_test, y_temp, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)
print(f"Train: {X_train.shape} Val: {X_val.shape} Test: {X_test.shape}")

X_train, X_val, X_test = map(lambda x: torch.tensor(x, dtype=torch.float32), [X_train, X_val, X_test])
y_train, y_val, y_test = map(lambda y: torch.tensor(y, dtype=torch.long), [y_train, y_val, y_test])

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=16, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=16, shuffle=False)

# ===========================================================
# Step 7. Transformer 模型
# ===========================================================
class PoseTransformer(nn.Module):
    def __init__(self, input_dim=96, num_classes=5, num_heads=4, num_layers=2, hidden_dim=128, dropout=0.2):
        super().__init__()
        self.pos_encoder = nn.Parameter(torch.randn(1, 100, input_dim))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(input_dim)
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = x + self.pos_encoder[:, :x.size(1), :]
        out = self.transformer(x)
        out = self.norm(out[:, -1, :])
        return self.fc(out)

# ===========================================================
# Step 8. 训练与验证
# ===========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PoseTransformer(input_dim=X_train.shape[2], num_classes=len(le.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

best_acc, patience, wait = 0.0, 8, 0

for epoch in range(1, 41):
    # ---------- Train ----------
    model.train()
    train_loss, train_correct = 0, 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
        train_correct += (out.argmax(1) == yb).sum().item()
    train_acc = train_correct / len(train_loader.dataset)

    # ---------- Validation ----------
    model.eval()
    val_loss, val_correct = 0, 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            val_loss += loss.item() * xb.size(0)
            val_correct += (out.argmax(1) == yb).sum().item()
    val_acc = val_correct / len(val_loader.dataset)

    # ---------- Test (new column) ----------
    test_correct = 0
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            test_correct += (out.argmax(1) == yb).sum().item()
    test_acc = test_correct / len(test_loader.dataset)

    print(f"Epoch {epoch:02d}/40 | "
          f"Train Loss: {train_loss/len(train_loader.dataset):.4f} Acc: {train_acc:.4f} | "
          f"Val Loss: {val_loss/len(val_loader.dataset):.4f} Acc: {val_acc:.4f} | "
          f"Test Acc: {test_acc:.4f} | LR: {scheduler.get_last_lr()}")

    scheduler.step()

    # ---------- Early stopping ----------
    if val_acc > best_acc:
        best_acc, wait = val_acc, 0
        torch.save(model.state_dict(), "data/transformer_pose5.pth")
    else:
        wait += 1
        if wait >= patience:
            print("⚠️ 提前停止：验证集不再提升。")
            break

print(f"✅ 最佳验证准确率: {best_acc:.4f}")

# ===========================================================
# Step 9. 测试阶段
# ===========================================================
state_dict = torch.load("data/transformer_pose5.pth", weights_only=True)
model.load_state_dict(state_dict)
model.eval()

y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        preds = out.argmax(1)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n🎯 最终测试集性能 (Test Set):")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))


原始 X shape: (144, 100, 32, 3) y shape: (144,)
原始类别分布: Counter({np.str_('p_stand'): 45, np.str_('a_walk'): 28, np.str_('p_sit'): 27, np.str_('Unknown'): 17, np.str_('t_bend'): 7, np.str_('p_lie'): 6, np.str_('p_bent'): 6, np.str_('t_straighten'): 4, np.str_('t_bed_turn'): 3, np.str_('t_stand_to_sit'): 1})
检测并移除 0 个坏样本（剩余 144 个），日志已保存到 data/bad_samples.txt
过滤后 X shape: (113, 100, 32, 3) y shape: (113,)
特征展开后: (113, 100, 96)
类别映射: {np.str_('a_walk'): 0, np.str_('p_lie'): 1, np.str_('p_sit'): 2, np.str_('p_stand'): 3, np.str_('t_bend'): 4}
过采样后类别分布: Counter({np.int64(0): 45, np.int64(1): 45, np.int64(2): 45, np.int64(3): 45, np.int64(4): 45})
Train: (135, 100, 96) Val: (45, 100, 96) Test: (45, 100, 96)
Epoch 01/40 | Train Loss: 1.6411 Acc: 0.2222 | Val Loss: 1.4576 Acc: 0.3111 | Test Acc: 0.2444 | LR: [0.001]
Epoch 02/40 | Train Loss: 1.4964 Acc: 0.3407 | Val Loss: 1.2613 Acc: 0.4667 | Test Acc: 0.4444 | LR: [0.001]
Epoch 03/40 | Train Loss: 1.4162 Acc: 0.3407 | Val Loss: 1.1601 Acc: 0.488

In [6]:
import os, numpy as np, torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report, confusion_matrix, f1_score
from sklearn.utils import resample
from collections import Counter

# ===========================================================
# Step 1. 数据加载
# ===========================================================
data_path = "data"
X = np.load(os.path.join(data_path, "X.npy"))
y = np.load(os.path.join(data_path, "y.npy"))

print(f"原始 X shape: {X.shape} y shape: {y.shape}")
print("原始类别分布:", Counter(y))

# ===========================================================
# Step 2. 过滤坏样本
# ===========================================================
def filter_invalid_samples(X, y, max_abs=10000.0, min_std=1e-6, log_path="data/bad_samples.txt"):
    valid_idx, bad_info = [], []
    for i in range(len(X)):
        xi = X[i]
        reason = None
        if np.isnan(xi).any() or np.isinf(xi).any():
            reason = "NaN/Inf"
        elif np.allclose(xi, 0):
            reason = "All zeros"
        elif np.std(xi) < min_std:
            reason = f"Low std ({np.std(xi):.2e})"
        elif np.any(np.abs(xi) > max_abs):
            reason = f"Out of range (max={np.max(np.abs(xi)):.1f})"

        if reason:
            bad_info.append(f"Sample {i}: {reason}")
        else:
            valid_idx.append(i)

    with open(log_path, "w", encoding="utf-8") as f:
        if bad_info:
            f.write("\n".join(bad_info))
        else:
            f.write("✅ 无坏样本\n")

    print(f"检测并移除 {len(bad_info)} 个坏样本（剩余 {len(valid_idx)} 个），日志已保存到 {log_path}")
    return X[valid_idx], y[valid_idx]

X, y = filter_invalid_samples(X, y)

# ===========================================================
# Step 3. 筛选目标类别
# ===========================================================
target_labels = ["p_sit", "p_stand", "a_walk", "p_lie", "t_bend"]
mask = np.isin(y, target_labels)
X, y = X[mask], y[mask]
print(f"过滤后 X shape: {X.shape} y shape: {y.shape}")

# ===========================================================
# Step 4. 特征展开
# ===========================================================
N, T, J, C = X.shape
X = X.reshape(N, T, J * C)
print(f"特征展开后: {X.shape}")

# ===========================================================
# Step 5. 标签编码 + 过采样
# ===========================================================
le = LabelEncoder()
y = le.fit_transform(y)
print("类别映射:", dict(zip(le.classes_, range(len(le.classes_)))))

X_bal, y_bal = [], []
for cls in np.unique(y):
    X_cls = X[y == cls]
    y_cls = y[y == cls]
    X_up, y_up = resample(X_cls, y_cls, replace=True, n_samples=max(Counter(y).values()), random_state=42)
    X_bal.append(X_up)
    y_bal.append(y_up)
X_res = np.vstack(X_bal)
y_res = np.hstack(y_bal)
print("过采样后类别分布:", Counter(y_res))

# ===========================================================
# Step 6. 数据划分 (Train / Val / Test)
# ===========================================================
X_temp, X_test, y_temp, y_test = train_test_split(X_res, y_res, test_size=0.2, random_state=42, stratify=y_res)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.25, random_state=42, stratify=y_temp)
print(f"Train: {X_train.shape} Val: {X_val.shape} Test: {X_test.shape}")

X_train, X_val, X_test = map(lambda x: torch.tensor(x, dtype=torch.float32), [X_train, X_val, X_test])
y_train, y_val, y_test = map(lambda y: torch.tensor(y, dtype=torch.long), [y_train, y_val, y_test])

train_loader = DataLoader(TensorDataset(X_train, y_train), batch_size=16, shuffle=True)
val_loader = DataLoader(TensorDataset(X_val, y_val), batch_size=16, shuffle=False)
test_loader = DataLoader(TensorDataset(X_test, y_test), batch_size=16, shuffle=False)

# ===========================================================
# Step 7. Transformer 模型定义
# ===========================================================
class PoseTransformer(nn.Module):
    def __init__(self, input_dim=96, num_classes=5, num_heads=4, num_layers=2, hidden_dim=128, dropout=0.2):
        super().__init__()
        self.pos_encoder = nn.Parameter(torch.randn(1, 100, input_dim))
        encoder_layer = nn.TransformerEncoderLayer(
            d_model=input_dim, nhead=num_heads, dim_feedforward=hidden_dim, dropout=dropout, batch_first=True
        )
        self.transformer = nn.TransformerEncoder(encoder_layer, num_layers=num_layers)
        self.norm = nn.LayerNorm(input_dim)
        self.fc = nn.Linear(input_dim, num_classes)

    def forward(self, x):
        x = x + self.pos_encoder[:, :x.size(1), :]
        out = self.transformer(x)
        out = self.norm(out[:, -1, :])
        return self.fc(out)

# ===========================================================
# Step 8. 训练与验证 (F1-based Early Stopping)
# ===========================================================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = PoseTransformer(input_dim=X_train.shape[2], num_classes=len(le.classes_)).to(device)
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

best_f1, patience, wait = 0.0, 8, 0

for epoch in range(1, 41):
    # ---------- Train ----------
    model.train()
    y_train_true, y_train_pred, train_loss = [], [], 0
    for xb, yb in train_loader:
        xb, yb = xb.to(device), yb.to(device)
        optimizer.zero_grad()
        out = model(xb)
        loss = criterion(out, yb)
        loss.backward()
        optimizer.step()
        train_loss += loss.item() * xb.size(0)
        y_train_true.extend(yb.cpu().numpy())
        y_train_pred.extend(out.argmax(1).cpu().numpy())
    train_f1 = f1_score(y_train_true, y_train_pred, average='macro')

    # ---------- Validation ----------
    model.eval()
    y_val_true, y_val_pred, val_loss = [], [], 0
    with torch.no_grad():
        for xb, yb in val_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            loss = criterion(out, yb)
            val_loss += loss.item() * xb.size(0)
            y_val_true.extend(yb.cpu().numpy())
            y_val_pred.extend(out.argmax(1).cpu().numpy())
    val_f1 = f1_score(y_val_true, y_val_pred, average='macro')

    # ---------- Test ----------
    y_test_true, y_test_pred = [], []
    with torch.no_grad():
        for xb, yb in test_loader:
            xb, yb = xb.to(device), yb.to(device)
            out = model(xb)
            y_test_true.extend(yb.cpu().numpy())
            y_test_pred.extend(out.argmax(1).cpu().numpy())
    test_f1 = f1_score(y_test_true, y_test_pred, average='macro')

    print(f"Epoch {epoch:02d}/40 | Train F1: {train_f1:.4f} | Val F1: {val_f1:.4f} | Test F1: {test_f1:.4f} | LR: {scheduler.get_last_lr()}")

    scheduler.step()

    # ---------- Early stopping based on F1 ----------
    if val_f1 > best_f1:
        best_f1, wait = val_f1, 0
        torch.save(model.state_dict(), "data/transformer_pose5_f1best.pth")
    else:
        wait += 1
        if wait >= patience:
            print("⚠️ Early stop")
            break

print(f"Best F1: {best_f1:.4f}")

# ===========================================================
# Step 9. 测试阶段评估
# ===========================================================
state_dict = torch.load("data/transformer_pose5_f1best.pth", weights_only=True)
model.load_state_dict(state_dict)
model.eval()

y_true, y_pred = [], []
with torch.no_grad():
    for xb, yb in test_loader:
        xb, yb = xb.to(device), yb.to(device)
        out = model(xb)
        preds = out.argmax(1)
        y_true.extend(yb.cpu().numpy())
        y_pred.extend(preds.cpu().numpy())

print("\n🎯 最终测试集性能 (Test Set):")
print(classification_report(y_true, y_pred, target_names=le.classes_, zero_division=0))
print("混淆矩阵:")
print(confusion_matrix(y_true, y_pred))


原始 X shape: (144, 100, 32, 3) y shape: (144,)
原始类别分布: Counter({np.str_('p_stand'): 45, np.str_('a_walk'): 28, np.str_('p_sit'): 27, np.str_('Unknown'): 17, np.str_('t_bend'): 7, np.str_('p_lie'): 6, np.str_('p_bent'): 6, np.str_('t_straighten'): 4, np.str_('t_bed_turn'): 3, np.str_('t_stand_to_sit'): 1})
检测并移除 0 个坏样本（剩余 144 个），日志已保存到 data/bad_samples.txt
过滤后 X shape: (113, 100, 32, 3) y shape: (113,)
特征展开后: (113, 100, 96)
类别映射: {np.str_('a_walk'): 0, np.str_('p_lie'): 1, np.str_('p_sit'): 2, np.str_('p_stand'): 3, np.str_('t_bend'): 4}
过采样后类别分布: Counter({np.int64(0): 45, np.int64(1): 45, np.int64(2): 45, np.int64(3): 45, np.int64(4): 45})
Train: (135, 100, 96) Val: (45, 100, 96) Test: (45, 100, 96)
Epoch 01/40 | Train F1: 0.2879 | Val F1: 0.1747 | Test F1: 0.1781 | LR: [0.001]
Epoch 02/40 | Train F1: 0.2618 | Val F1: 0.3366 | Test F1: 0.3351 | LR: [0.001]
Epoch 03/40 | Train F1: 0.3540 | Val F1: 0.4083 | Test F1: 0.3851 | LR: [0.001]
Epoch 04/40 | Train F1: 0.3212 | Val F1: 0.4557 | Te

In [1]:
import os, math, pickle, json, warnings
from collections import Counter, defaultdict

import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
from sklearn.metrics import classification_report, confusion_matrix

warnings.filterwarnings("ignore")

# =========================
# 0) 基本配置
# =========================
PICKLE_FILES = [
    "data/09_SY.pickle",
    "data/16_GZ.pickle",
    "data/17_JP.pickle",
]

SAVE_X = "data/external_X.npy"
SAVE_Y = "data/external_y.npy"

# 与训练时保持一致
T = 100
STRIDE = 25
NUM_JOINTS = 32
NUM_CH = 3
ROOT_J = 0                    # 根关节索引（据你的骨架定义可调整）
MAX_ABS_Z = 5.0               # 简单极值过滤
POSE_NDF_MIN = 0.10           # 可用 PoseNDF 下限阈值（若值可用）
REQUIRE_SINGLE_PERSON = False # 若需要只保留单人帧可改为 True

# 仅保留 5 个动作
KEEP_LABELS = ["a_walk", "p_lie", "p_sit", "p_stand", "t_bend"]
LABEL2ID = {k:i for i,k in enumerate(KEEP_LABELS)}

# =========================
# 1) 标签映射（兼容同义词）
# =========================
SYNONYMS = {
    "a_walk":       ["a_walk", "walk", "walking"],
    "p_lie":        ["p_lie", "lying", "lay", "laying"],
    "p_sit":        ["p_sit", "sitting", "sit"],
    "p_stand":      ["p_stand", "stand", "standing"],
    "t_bend":       ["t_bend", "bend", "bending", "p_bent"],  # 有时“p_bent”被当姿态/短动作
}

def map_activities_to_label(acts_dict):
    """
    从 activities 字典中找出与 5 类最接近的键。
    策略：
      1) 直接命中 KEEP_LABELS
      2) 命中同义词表
      3) 若 acts_dict 是 {label:score}，选分最高的且能映射的
    找不到则返回 None（该帧跳过）。
    """
    if not isinstance(acts_dict, dict) or len(acts_dict) == 0:
        return None

    # 先尝试直接命中
    keys_lower = {str(k).lower(): v for k, v in acts_dict.items()}

    # 直接命中
    for lbl in KEEP_LABELS:
        if lbl in keys_lower:
            return lbl

    # 同义词命中
    for lbl, syns in SYNONYMS.items():
        for s in syns:
            if s in keys_lower:
                return lbl

    # 选择一个“最像”的：分数最高且能通过同义词表匹配
    # 假设 values 是分数/概率/布尔 (True=1)
    try:
        candidates = sorted(keys_lower.items(), key=lambda kv: float(kv[1]), reverse=True)
    except:
        candidates = list(keys_lower.items())

    for k, _ in candidates:
        for lbl, syns in SYNONYMS.items():
            if any(s in k for s in syns):
                return lbl

    return None


# =========================
# 2) 坏姿态判定
# =========================
def is_bad_pose(arr32x3, pose_ndf=None):
    """简单坏姿态过滤：NaN/零方差/极值/PoseNDF过低"""
    if arr32x3.shape != (NUM_JOINTS, NUM_CH):
        return True
    if not np.isfinite(arr32x3).all():
        return True
    # 零方差（整骨架冻结）
    if np.allclose(arr32x3.std(axis=0), 0, atol=1e-6):
        return True
    # 简单极值过滤（可改为按分位数）
    if np.max(np.abs(arr32x3[:,2])) > MAX_ABS_Z:
        return True
    # PoseNDF（若提供）
    if pose_ndf is not None:
        try:
            score = float(pose_ndf)
            if score < POSE_NDF_MIN:
                return True
        except:
            pass
    return False


# =========================
# 3) 逐文件解析 -> 逐帧提取 (32,3) & label
# =========================
def extract_frames_from_pickle(path):
    with open(path, "rb") as f:
        data = pickle.load(f)

    frames_pose = []
    frames_label = []

    skipped_no3d = 0
    skipped_bad = 0
    mapped = 0

    for rec in data:
        # 3D pose
        p3d = rec.get("poses3d", None)
        if p3d is None or np.size(p3d) != 128:
            skipped_no3d += 1
            continue
        p3d = np.asarray(p3d).reshape(32, 4)[:, :3]  # 取 (32,3)

        # PoseNDF（可能是 [[x]]、None）
        pndf = rec.get("PoseNDF_score3d", None)
        if isinstance(pndf, (list, np.ndarray)) and np.size(pndf) >= 1:
            pndf_val = np.asarray(pndf).flatten()[0]
        else:
            pndf_val = None

        # 单人约束（可选）
        if REQUIRE_SINGLE_PERSON:
            n_people = rec.get("num_people", None)
            try:
                if int(n_people) != 1:
                    continue
            except:
                pass

        # 活动标签
        acts = rec.get("activities", {})
        lbl = map_activities_to_label(acts)

        # 筛掉非 5 类
        if lbl is None or lbl not in KEEP_LABELS:
            continue

        # 坏姿态过滤
        if is_bad_pose(p3d, pose_ndf=pndf_val):
            skipped_bad += 1
            continue

        frames_pose.append(p3d)
        frames_label.append(lbl)
        mapped += 1

    summary = {
        "total": len(data),
        "usable": mapped,
        "skipped_no3d": skipped_no3d,
        "skipped_badpose": skipped_bad,
        "label_dist": dict(Counter(frames_label)),
    }
    return np.array(frames_pose, dtype=np.float32), frames_label, summary


# =========================
# 4) 滑窗成序列 (N, T, 32, 3)
# =========================
def build_sequences(poses_list, labels_list, T=100, stride=25):
    """
    poses_list: 连续帧的 (32,3)；labels_list: 每帧的标签字符串
    策略：窗口内标签以“多数表决”为主（或中间帧），这里用“多数表决”
    """
    X_seq, y_seq = [], []
    n = len(poses_list)
    for start in range(0, max(n - T + 1, 0), stride):
        end = start + T
        block = poses_list[start:end]
        labs  = labels_list[start:end]
        if len(block) < T:
            break

        # 简单归一：根关节去中心化（与训练一致）
        block = block.copy()  # (T, 32, 3)
        block -= block[:, [ROOT_J], :]

        # 标签多数表决
        maj = Counter(labs).most_common(1)[0][0]

        X_seq.append(block)
        y_seq.append(LABEL2ID[maj])

    return np.array(X_seq, dtype=np.float32), np.array(y_seq, dtype=np.int64)


# =========================
# 5) 读取三文件 -> 汇总 -> 滑窗 -> 保存
# =========================
all_X_frames, all_y_frames = [], []
summaries = {}

for p in PICKLE_FILES:
    Xf, Yf, summ = extract_frames_from_pickle(p)
    summaries[os.path.basename(p)] = summ
    all_X_frames.append(Xf)
    all_y_frames += Yf  # 注意：labels 是字符串
    print(f"[{os.path.basename(p)}] {summ}")

if len(all_X_frames) == 0 or sum(map(len, all_X_frames)) == 0:
    raise RuntimeError("没有可用帧，请检查 PoseNDF 阈值/活动映射/文件内容。")

# 连起来（默认按文件内部时间顺序已是连续的；不同文件之间不连窗）
X_total, y_total = [], []
offset = 0
for i, p in enumerate(PICKLE_FILES):
    Xf, Yf, _ = extract_frames_from_pickle(p)
    if len(Xf) == 0:
        continue
    Xseq, yseq = build_sequences(Xf, Yf, T=T, stride=STRIDE)
    print(f"Build sequences from {os.path.basename(p)} -> {Xseq.shape}")
    if len(Xseq):
        X_total.append(Xseq)
        y_total.append(yseq)

if len(X_total) == 0:
    raise RuntimeError("滑窗后没有可用样本，请降低 T 或 stride，或放宽过滤规则。")

X = np.concatenate(X_total, axis=0)
y = np.concatenate(y_total, axis=0)
print("\n=== External dataset (after windowing) ===")
print("X shape:", X.shape, "y shape:", y.shape)
print("Label dist:", {KEEP_LABELS[i]: int(n) for i, n in Counter(y).items()})

np.save(SAVE_X, X)
np.save(SAVE_Y, y)
print(f"Saved to {SAVE_X} / {SAVE_Y}")

# =========================
# 6) 评测：加载你之前的 Transformer 模型并在外部数据上测试
#   （下面是与你之前版本一致的轻量 Transformer 分类头）
# =========================

class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=5000):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/d_model))
        pe[:, 0::2] = torch.sin(pos * div)
        pe[:, 1::2] = torch.cos(pos * div)
        self.register_buffer('pe', pe.unsqueeze(0))  # (1, L, d_model)
    def forward(self, x):
        # x: (B, L, d_model)
        L = x.size(1)
        return x + self.pe[:, :L, :]

class TinyTransformer(nn.Module):
    def __init__(self, num_classes=5, d_model=128, nhead=4, num_layers=2, dim_ff=256, dropout=0.1):
        super().__init__()
        in_dim = NUM_JOINTS * NUM_CH
        self.proj = nn.Linear(in_dim, d_model)
        self.pe = PositionalEncoding(d_model)
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead, dim_feedforward=dim_ff, dropout=dropout, batch_first=True)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.cls = nn.Sequential(
            nn.LayerNorm(d_model),
            nn.Linear(d_model, num_classes)
        )
    def forward(self, x):
        # x: (B, T, 32, 3)
        B, T, J, C = x.shape
        x = x.reshape(B, T, J*C)
        x = self.proj(x)
        x = self.pe(x)
        h = self.encoder(x)          # (B, T, d_model)
        h = h.mean(dim=1)            # 全局平均
        return self.cls(h)

# 载入外部数据
X_ext = np.load(SAVE_X)
y_ext = np.load(SAVE_Y)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

# 模型定义（参数需与你训练时一致；如你保存的权重路径不同，请修改）
model = TinyTransformer(num_classes=len(KEEP_LABELS))
ckpt_path = "data/transformer_pose5.pth"  # 你之前保存的 transformer 权重（若名称不同改这里）
if os.path.exists(ckpt_path):
    state = torch.load(ckpt_path, map_location="cpu")
    try:
        model.load_state_dict(state, strict=True)
    except:
        # 兼容只存 state_dict 的情况
        model.load_state_dict(state.get("model", state))
    print(f"Loaded weights: {ckpt_path}")
else:
    print(f"⚠ 找不到 {ckpt_path}，将使用随机初始化权重，仅做流程演示。")

model.to(device).eval()

# 推理
with torch.no_grad():
    logits_all, y_all = [], []
    BS = 64
    for i in range(0, len(X_ext), BS):
        xb = torch.tensor(X_ext[i:i+BS], dtype=torch.float32, device=device)
        out = model(xb)
        logits_all.append(out.cpu())
        y_all.append(torch.tensor(y_ext[i:i+BS]))
    logits_all = torch.cat(logits_all, dim=0)
    y_all = torch.cat(y_all, dim=0)
    pred = logits_all.argmax(dim=1).numpy()

# 指标
names = KEEP_LABELS
print("\n=== External Test Report ===")
print(classification_report(y_all, pred, target_names=names, digits=2))
print("Confusion matrix:\n", confusion_matrix(y_all, pred))


[09_SY.pickle] {'total': 47199, 'usable': 0, 'skipped_no3d': 23858, 'skipped_badpose': 0, 'label_dist': {}}
[16_GZ.pickle] {'total': 33409, 'usable': 0, 'skipped_no3d': 16439, 'skipped_badpose': 0, 'label_dist': {}}
[17_JP.pickle] {'total': 36546, 'usable': 0, 'skipped_no3d': 15128, 'skipped_badpose': 0, 'label_dist': {}}


RuntimeError: 没有可用帧，请检查 PoseNDF 阈值/活动映射/文件内容。

In [4]:
import pickle, os
from collections import Counter

label_keys = ["physicalstate","PhysicalState","phys_state","physState"]

def extract_all_labels(pickle_path, max_frames=20000):
    with open(pickle_path, "rb") as f:
        data = pickle.load(f)

    print(f"\n====== Checking labels in {os.path.basename(pickle_path)} ======")
    total = len(data)
    found_labels = Counter()
    found_raw_texts = Counter()
    frames_with_label = 0
    frames_checked = 0

    for rec in data[:max_frames]:
        frames_checked += 1
        label = None
        # ---- 1) 顶层 physicalstate ----
        for k in label_keys:
            if k in rec:
                label = rec[k]
                break
        # ---- 2) activities 下 physicalstate ----
        if label is None:
            acts = rec.get("activities", {})
            for k in label_keys:
                if k in acts:
                    label = acts[k]
                    break
            # 再检查 activities 里的其他 key 是否为 label
            if label is None:
                for v in acts.values():
                    if v not in [None, {}, []]:
                        label = v
                        break

        if label is not None:
            frames_with_label += 1

            # 统计原始标签文本（字符串 or dict / list 展开）
            if isinstance(label, dict):
                for vv in label.values():
                    found_raw_texts[str(vv)] += 1
            elif isinstance(label, (list, tuple)):
                for vv in label:
                    found_raw_texts[str(vv)] += 1
            else:
                found_raw_texts[str(label)] += 1

    print(f"总帧数: {total}")
    print(f"检查帧数: {frames_checked}")
    print(f"有标签的帧: {frames_with_label}")
    print("📌 找到的所有 raw label 文本（最多显示前 30 个）:")
    for k, v in found_raw_texts.most_common(30):
        print(f"  {k}: {v}")

    return found_raw_texts


files = [
    "data/09_SY.pickle",
    "data/16_GZ.pickle",
    "data/17_JP.pickle"
]

all_outputs = []
for f in files:
    out = extract_all_labels(f, max_frames=20000)
    all_outputs.append(out)



====== Checking labels in 09_SY.pickle ======
总帧数: 47199
检查帧数: 20000
有标签的帧: 6098
📌 找到的所有 raw label 文本（最多显示前 30 个）:
  p_lie: 2304
  p_sit: 942
  t_bed_turn: 588
  p_situp: 509
  idle: 415
  t_sit_to_lie: 361
   floor: 341
  t_situp_to_sit: 331
  signal: 155
  t_lie_to_situp: 151
  operate_object: 139
  get_object: 109
  place_object: 94
   sofa: 1
   t_situp_to_sit: 1

====== Checking labels in 16_GZ.pickle ======
总帧数: 33409
检查帧数: 20000
有标签的帧: 15224
📌 找到的所有 raw label 文本（最多显示前 30 个）:
  p_stand: 6557
   kitchen_table: 3155
  a_walk: 2348
  p_sit: 1620
   stool: 1536
  p_bent: 1308
  p_situp: 810
  p_lie: 741
   bed: 673
  t_bed_turn: 589
   chair: 257
  t_straighten: 251
  t_bend: 250
  t_stand_to_sit: 196
  t_sit_to_stand: 179
  t_lie_to_situp: 176
  t_situp_to_sit: 121
  t_sit_to_lie: 76
   p_stand: 8
   t_bed_turn: 4
   t_bend: 3
   t_straighten: 3
   t_stand_to_sit: 2
   t_sit_to_stand: 2
  idle: 1
  bed: 1
   t_sit_to_lie: 1

====== Checking labels in 17_JP.pickle ======
总帧数: 36546


In [8]:
import os, re, pickle, math, warnings
from collections import Counter, defaultdict
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F

# =============================
# 1) 可调参数（放宽+兜底）
# =============================
PICKLES = ["data/16_GZ.pickle", "data/17_JP.pickle"]   # 你也可以加上 "data/09_SY.pickle"
KEEP = ["a_walk","p_sit","p_stand","p_lie","t_bend"]
LABEL2ID = {k:i for i,k in enumerate(KEEP)}
WINDOW = 30         # 每窗长度（放宽）
STRIDE = 10         # 步长（更密集）
MAJORITY = 0.40     # 主标签占比阈值（放宽）
MIN_SEG_LEN = 10    # 连续同类片段最少帧（放宽）
CONF3D_MIN = 0.20   # PoseNDF 3D 阈值（放宽）
MISS_TOL = 0.25     # 允许25%关节为 NaN（放宽）
SINGLE_PERSON = True  # 先限制单人；采不到再改 False

SAVE_X = "data/X_test_pose5.npy"
SAVE_Y = "data/y_test_pose5.npy"

# 可选：评测已训练 Transformer 模型
EVAL_MODEL = True
MODEL_PATH = "data/transformer_top5_improved.pth"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

# =============================
# 2) 标签规范化（合并同义/过程态→静态）
# =============================
def norm_label(s: str):
    if s is None: return None
    s = s.strip().lower()
    # 常见噪声或无效
    if s in ["idle","undefined","signal","operate_object","get_object","place_object"]:
        return None
    # 家具/场景词忽略
    if s in ["bed","chair","stool","kitchen_table","sofa","floor","p_stand"]:  # 注意 'p_stand'（带空格）也可能出现在你的数据里
        return None
    # 过程态映射
    map_proc = {
        "t_sit_to_stand":"p_stand",
        "t_stand_to_sit":"p_sit",
        "t_sit_to_lie":"p_lie",
        "t_lie_to_sit":"p_sit",       # 若你的数据有这个
        "t_lie_to_situp":"p_sit",
        "t_situp_to_sit":"p_sit",
        "t_straighten":"t_bend"       # 视作弯腰类
    }
    if s in map_proc: s = map_proc[s]
    # 仅保留我们 5 类
    if s in KEEP: return s
    # 其他别名处理
    alias = {
        "walk":"a_walk",
        "stand":"p_stand",
        "sit":"p_sit",
        "lie":"p_lie",
        "bend":"t_bend"
    }
    return alias.get(s, None)

# =============================
# 3) 从单帧 dict 提取一帧 96 维特征（32关节×xyz）
# =============================
SELECT_32 = None  # 如果你的关节数>32可在此挑选索引；默认取前32并截/零填
def frame_to_feat(d):
    # 取第一人的3D关键点（你的数据通常是 list/array，长度=关节数）
    poses3d = d.get("poses3d", None)
    if poses3d is None or len(poses3d)==0:
        return None, 1.0  # 无3D，缺失比例=1
    # 兼容各种存储：可能是 list/array/np.ndarray；取第一人或第一组
    p = poses3d[0] if isinstance(poses3d, (list,tuple)) else poses3d
    p = np.array(p)
    # 形状兼容：(J,3) or (…,J,3)
    if p.ndim==1: 
        return None, 1.0
    if p.ndim>2:
        # 取最后两维为 (J,3)
        p = p.reshape(-1,3)
    # 统一到 32 关节
    J = p.shape[0]
    if J >= 32:
        if SELECT_32 is None:
            p = p[:32]
        else:
            idx = np.array(SELECT_32, dtype=int)
            p = p[idx]
    else:
        pad = np.zeros((32-J,3), dtype=float)
        p = np.vstack([p, pad])
    # 缺失比例
    miss = np.isnan(p).mean()
    # NaN→0
    p = np.nan_to_num(p, nan=0.0)
    feat = p.reshape(-1).astype(np.float32)  # (96,)
    return feat, float(miss)

def frame_conf3d(d):
    sc = d.get("PoseNDF_score3d", None)
    if sc is None:
        return None
    sc = np.array(sc)
    try:
        return float(sc.squeeze())
    except:
        return None

def frame_label(d):
    # activities 可能是 dict：{label:prob} 或 {label:1}
    acts = d.get("activities", {})
    cand = []
    for k,v in (acts.items() if isinstance(acts,dict) else []):
        lab = norm_label(str(k))
        if lab: cand.append(lab)
    if not cand: 
        return None
    # 多标签时取第一个可用
    for lab in cand:
        if lab in KEEP:
            return lab
    return None

def frame_num_people(d):
    # 优先用num_people，否则用 poses3d 的人数估计
    n = d.get("num_people", None)
    if isinstance(n, (int,np.integer)): 
        return int(n)
    poses3d = d.get("poses3d", None)
    if poses3d is None: 
        return 0
    if isinstance(poses3d, list):
        # 可能是 [ (J,3), (J,3) ] 或 [J,3]
        if len(poses3d)>0 and isinstance(poses3d[0], (list,tuple,np.ndarray)) and np.array(poses3d[0]).ndim==2:
            return len(poses3d)
        else:
            return 1
    return 1

# =============================
# 4) 从一个 pickle 中抽窗
# =============================
def extract_from_one_pickle(path):
    with open(path,"rb") as f:
        data = pickle.load(f)
    assert isinstance(data, list), f"{path} 不是 list"
    Nf = len(data)
    # 每帧：质量筛选 + 标签
    frames = []
    labels = []
    for d in data:
        lab = frame_label(d)
        if lab is None:
            frames.append(None); labels.append(None); continue
        if SINGLE_PERSON and frame_num_people(d)!=1:
            frames.append(None); labels.append(None); continue
        feat, miss = frame_to_feat(d)
        if feat is None or miss > MISS_TOL:
            frames.append(None); labels.append(None); continue
        conf = frame_conf3d(d)
        if conf is not None and conf < CONF3D_MIN:
            frames.append(None); labels.append(None); continue
        frames.append(feat); labels.append(lab)

    frames = np.array(frames, dtype=object)
    # 滑窗（多数占比）
    X, y = [], []
    for s in range(0, Nf-WINDOW+1, STRIDE):
        win_feats = frames[s:s+WINDOW]
        win_labs  = labels[s:s+WINDOW]
        if any(v is None for v in win_feats):
            continue
        # 多数占比
        c = Counter([L for L in win_labs if L is not None])
        if not c: 
            continue
        maj, cnt = c.most_common(1)[0]
        if maj in KEEP and cnt/len(win_labs) >= MAJORITY:
            X.append(np.stack(win_feats, axis=0))     # (T,96)
            y.append(LABEL2ID[maj])

    # 兜底：如果没采到，按标签中心截窗（每标签每隔 K 帧取一个中心）
    if len(X)==0:
        centers = defaultdict(list)
        for i,lab in enumerate(labels):
            if lab in KEEP and frames[i] is not None:
                centers[lab].append(i)
        for lab, idxs in centers.items():
            for c in idxs[::WINDOW]:  # 适当抽稀
                s = max(0, c - WINDOW//2)
                e = s + WINDOW
                if e <= Nf and all(frames[s:e][j] is not None for j in range(WINDOW)):
                    X.append(np.stack(frames[s:e], axis=0))
                    y.append(LABEL2ID[lab])

    return np.array(X, dtype=np.float32), np.array(y, dtype=np.int64)

def build_test_from_pickles(paths):
    Xs, ys = [], []
    stat = []
    for p in paths:
        if not os.path.exists(p):
            print(f"跳过（不存在）：{p}")
            continue
        Xp, yp = extract_from_one_pickle(p)
        stat.append( (os.path.basename(p), len(yp), Counter(yp.tolist())) )
        if len(yp)>0:
            Xs.append(Xp); ys.append(yp)
        print(f"{os.path.basename(p)} 产出窗口: X={Xp.shape} y={yp.shape} 标签分布={Counter(yp.tolist())}")
    if Xs:
        X = np.concatenate(Xs, axis=0)
        y = np.concatenate(ys, axis=0)
    else:
        X = np.zeros((0, WINDOW, 96), dtype=np.float32)
        y = np.zeros((0,), dtype=np.int64)
    return X, y, stat

# =============================
# 5) （可选）Transformer 评测（与之前一致）
# =============================
class PosEnc1D(nn.Module):
    def __init__(self, d_model, max_len=512):
        super().__init__()
        pe = torch.zeros(max_len, d_model)
        pos = torch.arange(0, max_len, dtype=torch.float32).unsqueeze(1)
        div = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/d_model))
        pe[:,0::2] = torch.sin(pos*div)
        pe[:,1::2] = torch.cos(pos*div)
        self.register_buffer('pe', pe.unsqueeze(0))  # (1, max_len, d_model)
    def forward(self, x):
        # x: (B,T,D)
        T = x.size(1)
        return x + self.pe[:, :T, :]

class TinyTransformer(nn.Module):
    def __init__(self, in_dim=96, d_model=128, nhead=4, num_layers=2, num_classes=5, dim_ff=256, dropout=0.1):
        super().__init__()
        self.inp = nn.Linear(in_dim, d_model)
        self.pe  = PosEnc1D(d_model, max_len=1024)
        enc_layer = nn.TransformerEncoderLayer(d_model=d_model, nhead=nhead,
                                               dim_feedforward=dim_ff, dropout=dropout,
                                               batch_first=True, norm_first=True)
        self.encoder = nn.TransformerEncoder(enc_layer, num_layers=num_layers)
        self.cls = nn.Linear(d_model, num_classes)
    def forward(self, x):
        # x: (B,T,96)
        h = self.inp(x)
        h = self.pe(h)
        h = self.encoder(h)
        h = h.mean(dim=1)    # GAP
        return self.cls(h)

@torch.no_grad()
def evaluate(model, device, X, y):
    model.eval()
    loader_idx = np.arange(len(y))
    probs = []
    for i in range(0, len(loader_idx), 64):
        idx = loader_idx[i:i+64]
        xb  = torch.tensor(X[idx], dtype=torch.float32, device=device)
        out = model(xb)
        p = F.softmax(out, dim=-1).cpu().numpy()
        probs.append(p)
    probs = np.concatenate(probs, axis=0)
    y_pred = probs.argmax(axis=1)
    from sklearn.metrics import classification_report, confusion_matrix
    print("\n📊 测试集性能：")
    print(classification_report(y, y_pred, target_names=KEEP, digits=2))
    print("混淆矩阵:\n", confusion_matrix(y, y_pred))

def load_model(num_classes=5):
    model = TinyTransformer(num_classes=num_classes)
    if os.path.exists(MODEL_PATH):
        ckpt = torch.load(MODEL_PATH, map_location="cpu")
        missing, unexpected = model.load_state_dict(ckpt, strict=False)
        if missing:   print("⚠️ Missing keys:", missing)
        if unexpected:print("⚠️ Unexpected keys:", unexpected)
        print(f"✅ 已加载模型权重: {MODEL_PATH}")
    else:
        print("⚠️ 未找到模型权重，使用随机初始化模型。")
    model.to(DEVICE)
    return model

# =============================
# 6) 主流程
# =============================
if __name__ == "__main__":
    print("▶ 从新 pickle 构造 5 类测试集（宽松+兜底）...")
    X_test, y_test, stat = build_test_from_pickles(PICKLES)
    print("各文件产出统计：", stat)
    print("完整测试集: X_test=", X_test.shape, " y_test=", y_test.shape)
    print("标签分布:", Counter(y_test.tolist()))
    # 保存
    np.save(SAVE_X, X_test)
    np.save(SAVE_Y, y_test)
    print(f"💾 已保存到: {SAVE_X} / {SAVE_Y}")

    if EVAL_MODEL and len(y_test)>0:
        model = load_model(num_classes=len(KEEP))
        evaluate(model, DEVICE, X_test, y_test)
    elif EVAL_MODEL:
        print("⚠️ 测试集为空，跳过评测。")


▶ 从新 pickle 构造 5 类测试集（宽松+兜底）...
16_GZ.pickle 产出窗口: X=(0,) y=(0,) 标签分布=Counter()
17_JP.pickle 产出窗口: X=(0,) y=(0,) 标签分布=Counter()
各文件产出统计： [('16_GZ.pickle', 0, Counter()), ('17_JP.pickle', 0, Counter())]
完整测试集: X_test= (0, 30, 96)  y_test= (0,)
标签分布: Counter()
💾 已保存到: data/X_test_pose5.npy / data/y_test_pose5.npy
⚠️ 测试集为空，跳过评测。


In [9]:
import pickle, numpy as np

def check_pose3d(path):
    with open(path, "rb") as f:
        data = pickle.load(f)

    total = len(data)
    valid = 0
    valid_idx = []

    for i, d in enumerate(data):
        p3d = d.get("poses3d", None)

        if p3d is None:
            continue

        arr = np.array(p3d)
        # 检查是否包含真正的 (J,3)
        if arr.ndim == 3 and arr.shape[-1] == 3:
            valid += 1
            valid_idx.append(i)

    print("\n====", path, "====")
    print("总帧:", total)
    print("有效 3D pose 帧:", valid)

    # 检查连续段
    seqs = []
    if valid_idx:
        seq = [valid_idx[0]]
        for i in range(1, len(valid_idx)):
            if valid_idx[i] == valid_idx[i-1] + 1:
                seq.append(valid_idx[i])
            else:
                seqs.append(seq)
                seq = [valid_idx[i]]
        seqs.append(seq)

    seq_lens = [len(s) for s in seqs]
    print("连续 3D 段数量:", len(seqs))
    print("最长连续 3D 段长度:", max(seq_lens) if seq_lens else 0)
    print("所有段长度（前10）:", seq_lens[:10])

check_pose3d("data/16_GZ.pickle")
check_pose3d("data/17_JP.pickle")



==== data/16_GZ.pickle ====
总帧: 33409
有效 3D pose 帧: 0
连续 3D 段数量: 0
最长连续 3D 段长度: 0
所有段长度（前10）: []

==== data/17_JP.pickle ====
总帧: 36546
有效 3D pose 帧: 0
连续 3D 段数量: 0
最长连续 3D 段长度: 0
所有段长度（前10）: []


In [10]:
import pickle, numpy as np

def check_pose3d(path):
    with open(path, "rb") as f:
        data = pickle.load(f)

    total = len(data)
    valid = 0
    valid_idx = []

    for i, d in enumerate(data):
        p3d = d.get("poses3d", None)

        if p3d is None:
            continue

        arr = np.array(p3d)
        # 检查是否包含真正的 (J,3)
        if arr.ndim == 3 and arr.shape[-1] == 3:
            valid += 1
            valid_idx.append(i)

    print("\n====", path, "====")
    print("总帧:", total)
    print("有效 3D pose 帧:", valid)

    # 检查连续段
    seqs = []
    if valid_idx:
        seq = [valid_idx[0]]
        for i in range(1, len(valid_idx)):
            if valid_idx[i] == valid_idx[i-1] + 1:
                seq.append(valid_idx[i])
            else:
                seqs.append(seq)
                seq = [valid_idx[i]]
        seqs.append(seq)

    seq_lens = [len(s) for s in seqs]
    print("连续 3D 段数量:", len(seqs))
    print("最长连续 3D 段长度:", max(seq_lens) if seq_lens else 0)
    print("所有段长度（前10）:", seq_lens[:10])

check_pose3d("data/19_MM.pickle")




==== data/19_MM.pickle ====
总帧: 34001
有效 3D pose 帧: 0
连续 3D 段数量: 0
最长连续 3D 段长度: 0
所有段长度（前10）: []
